
# Data-driven lingerie shopping

## Massive data cleaning is performed in this notebook


This is a detailed analysis of [lingerie dataset](https://www.kaggle.com/PromptCloudHQ/innerwear-data-from-victorias-secret-and-others) published on kaggle. It was created by extracting data from the popular retail sites via PromptCloud's data extraction solutions in June, 2017 to July, 2017.


Websites and brands covered in this analysis:

* Victoria's Secret (official website and Amazon)<br>
* Wacoal aka Btemptd (official website, Amazon and Macy's <br>
* Calvin Klein (official website, Amazon and Macy's) <br>
* Hanky Panky (official website, Amazon and Macy's<br>
* Topshop USA<br>

I will happy to hear some remarks or suggestions and feel free to upvote if you like it :)

NB: There were records that had different prices on the same products: price adjustment to make higher when people buy the product a lot or make lower when they see that people are not buying it.

NB2: Cleaning of the data took most of the analysis time.

# Load data

In [1]:
import pandas as pd
import numpy as np
# to normalize the strings
import unicodedata
from IPython.display import display
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 1000)
import warnings
warnings.filterwarnings("ignore")

In [2]:
calvin_klein = pd.read_csv('data/calvinklein_com.csv')
victoria_sec = pd.read_csv('data/victoriassecret_com.csv')
hankypanky = pd.read_csv('data/hankypanky_com.csv')
amazon = pd.read_csv('data/amazon_com.csv')
topshop = pd.read_csv('data/us_topshop_com.csv')
macys = pd.read_csv('data/macys_com.csv')
wacoal = pd.read_csv('data/btemptd_com.csv')

In [3]:
calvin_klein.head(2)

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
0,logo cotton stretch thong,$13.00,$13.00,http://www.calvinklein.us/en/womens-clothing/women-three-for-panties/logo-cotton-stretch-thong-51703539,Calvin Klein,3 FOR 33 PANTY ESSENTIALS,Calvin Klein US,soft cotton stretch fabric and a metallic logo waistband in a minimal rear coverage style.,NaN,NaN,"[""cotton stretch thong panty"",""metallic elasticized logo waistband"",""minimal rear coverage"",""full cotton gusset"",""tonal topstitching"",""seaming details"",""machine wash"",""95% cotton 5% elastane"",""imported""]","s,m,l","s,m,l",BLACK
1,logo cotton stretch thong,$13.00,$13.00,http://www.calvinklein.us/en/womens-clothing/women-three-for-panties/logo-cotton-stretch-thong-51703539,Calvin Klein,3 FOR 33 PANTY ESSENTIALS,Calvin Klein US,soft cotton stretch fabric and a metallic logo waistband in a minimal rear coverage style.,NaN,NaN,"[""cotton stretch thong panty"",""metallic elasticized logo waistband"",""minimal rear coverage"",""full cotton gusset"",""tonal topstitching"",""seaming details"",""machine wash"",""95% cotton 5% elastane"",""imported""]","s,m,l","s,m,l",ASHFORD GREY


In [4]:
amazon.groupby('brand_name').size()

brand_name
Calvin-Klein                                                                                           7372 
Compression-Comfort                                                                                    1    
Creative-Motion                                                                                        1    
Fila                                                                                                   1    
Hanky-Panky                                                                                            582  
Lucky-Brand                                                                                            1    
Nintendo                                                                                               1    
Sexy-Hair                                                                                              1    
Victorias-Secret                                                                                       67   
Wacoal  

In [5]:
macys.groupby('brand_name').size()

brand_name
Calvin Klein                  14145
Calvin Klein Modern Cotton    1193 
Calvin Klein Performance      651  
Hanky Panky                   8133 
Vanity Fair                   2575 
Wacoal                        8890 
b.tempt'd                     69   
b.tempt'd by Wacoal           5241 
dtype: int64

In [6]:
macys_panky = macys[macys['brand_name'] == 'Hanky Panky']
macys_wacoal = macys[macys['brand_name'].isin(['Wacoal', "b.tempt'd", "b.tempt'd by Wacoal"])]
macys_klein = macys[macys['brand_name'].isin(['Calvin Klein', 'Calvin Klein Modern Cotton', 'Calvin Klein Performance'])]

In [7]:
amazon_klein = amazon[amazon['brand_name'] == 'Calvin-Klein']
amazon_panky = amazon[amazon['brand_name'] == 'Hanky-Panky']
amazon_vic_secret = amazon[amazon['brand_name'] == 'Victorias-Secret']
amazon_wacoal = amazon[amazon['brand_name'].isin(['WACOAL', 'b-temptd'])]

In [8]:
print(calvin_klein.shape)
print(victoria_sec.shape)
print(hankypanky.shape)
print(wacoal.shape)
print(topshop.shape)
print(amazon_klein.shape)
print(amazon_panky.shape)
print(amazon_vic_secret.shape)
print(amazon_wacoal.shape)
print(macys_klein.shape)
print(macys_panky.shape)
print(macys_wacoal.shape)


(4747, 14)
(453386, 14)
(35005, 14)
(3518, 14)
(3082, 14)
(7372, 14)
(582, 14)
(67, 14)
(3490, 14)
(15989, 14)
(8133, 14)
(14200, 14)


# 1. Find abnormalities

Prior analzsios it is important to understand the data and find abnormalities. In time series that could be that the timepoints should be increasing and shoult not contain gaps etc.
First, we will strip some important columns just in case the was some mistakes on the webpage.

In [9]:
def strip_columns(data):
    cols = list(set(data.columns.tolist()) - set(['review_count', 'rating', 'style_attributes']))
    for col in cols:
        data[col] = data[col].str.strip().str.lower()
    return data

In [10]:
calvin_klein = strip_columns(calvin_klein)
hankypanky = strip_columns(hankypanky)
victoria_sec = strip_columns(victoria_sec)
topshop = strip_columns(topshop)
wacoal = strip_columns(wacoal)

amazon_klein = strip_columns(amazon_klein)
amazon_panky = strip_columns(amazon_panky)
amazon_vic_secret = strip_columns(amazon_vic_secret)
amazon_wacoal = strip_columns(amazon_wacoal)

macys_klein = strip_columns(macys_klein)
macys_panky = strip_columns(macys_panky)
macys_wacoal = strip_columns(macys_wacoal)

## 1.1 Duplicated records
When scaping the pages we can get some duplicates records. Lets check if we have some in our dataset. 
Several reasons of the duplicated records:
1. If a scaper collects data every 10 minutes and we get the same webpage several times. Same records with have everything the same except for the rating and review count.
2. A product can belong to two different categories. In case of Calvin Klein panties can belong to a 'Panties' category but also to '3 FOR 33 PANTY ESSENTIALS', what results in different product_categories and sometimes in different pdp_url. (As for example in Hanky Panky) **We will change '3 FOR 33 PANTY ESSENTIALS' to be panties** for the simplicity. Hanky Panky will also be considered seperately.
3. When developers change some specifications on the website, we also get a duplicated record :). And those changes are very hard to find. The one that I found was in Hanky Panky. Developers changed the dopdown menu for the size. There are two same products with values for available size as ["Select", "One Size"] and ["One Size"]. I will just hope that there are no such cases in the other datasets. :)
4. Also, when content managers change some attributes(or description) of the product, we will again get duplicates. In Macy'sCalvin Klein the style attributes were changes, although everything else stayed the same. Calvin Klein original website had the same problems.
5. The could be the sampe products but with different mrp. It could be a mistake in the dataset or whatever. We will sort our sets according mrp and find duplicates by description and brand.

In order to get a better price (e.g. discounted price) we need to sort the datasets according to the price (ascending) and then find the duplicates according to all columns except for 'review_count', 'rating', 'pdp_url', 'product_category'. So, first we will convert prices into float.

Example of the same product with different pdp_urls:
http://www.hankypanky.com/thongs/30-pack-original-rise-thongs-in-jar.html 
http://www.hankypanky.com/collections/30-pack-original-rise-thongs-in-jar.html


Example of the 5th point:

In [11]:
calvin_klein[calvin_klein.pdp_url=='http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-ck-id-triangle-black'][['color','price','total_sizes','description','mrp','product_name']]

,color,price,total_sizes,description,mrp,product_name
4536,black,$32.00,"xs,s,m,l,xl","a ck id unlined triangle bra and string bikini set featuring a convertible design, cotton stretch fabric and a thin logo band revisiting cotton classic styling and exploring a new shape.",$32.00,ck id thin waistband unlined triangle bra
4537,black,$20.00,"xs,s,m,l,xl","a ck id unlined triangle bra and string bikini set featuring a convertible design, cotton stretch fabric and a thin logo band revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini


In [12]:

calvin_klein[calvin_klein.product_name=='ck id thin waistband string bikini'][['color','price','total_sizes','description','mrp','product_name']]

,color,price,total_sizes,description,mrp,product_name
404,black,$20.00,"xs,s,m,l,xl","a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini
405,white,$20.00,"xs,s,m,l,xl","a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini
406,transpink,$20.00,"xs,s,m,l,xl","a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini
407,simple grid print,$9.99,"xs,s,m,l,xl","a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini
659,black,$20.00,"xs,s,m,l,xl","a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini
660,white,$20.00,"xs,s,m,l,xl","a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini
661,transpink,$20.00,"xs,s,m,l,xl","a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini
662,simple grid print,$9.99,"xs,s,m,l,xl","a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini
1586,black,$20.00,"xs,s,m,l,xl","a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini
1587,white,$20.00,"xs,s,m,l,xl","a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",$20.00,ck id thin waistband string bikini


Developers change (dropdown menu) led to the duplicates:

In [13]:
display(hankypanky.loc[5])
display(hankypanky.loc[33555])

product_name        30 pack original rise thongs in lucite jar                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
mrp                 $615                                                                                                                 

product_name        30 pack original rise thongs in lucite jar                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
mrp                 $615                                                                                                                 

Content manager change led to:

In [14]:
macys_klein[(macys_klein['color']== 'black')&(macys_klein['product_name']== 'ck black endless silk lace robe qs5679')]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
338,ck black endless silk lace robe qs5679,$125.00,$125.00,http://www1.macys.com/shop/product/calvin-klein-ck-black-endless-silk-lace-robe-qs5679?id=4582655&categoryid=65739,calvin klein,women - lingerie & shapewear - designer lingerie,macys us,"experience the ultimate luxury with the soft, lightweight silk of this sensational robe from ck black endless by calvin klein.",NaN,NaN,"[""Open neckline and front with belt closure at waist"", ""Sleeveless with lace trim at arm openings"", ""Hits at mid-thigh"", ""Silk"", ""Hand wash"", ""Imported"", ""Web ID: 4582655""]","[""xs-s"", ""xs/s"", ""m-l"", ""m/l""]","[""xs/s"", ""m/l""]",black
4492,ck black endless silk lace robe qs5679,$125.00,$125.00,http://www1.macys.com/shop/product/calvin-klein-ck-black-endless-silk-lace-robe-qs5679?id=4582655&categoryid=65739,calvin klein,women - lingerie & shapewear - designer lingerie,macys us,"experience the ultimate luxury with the soft, lightweight silk of this sensational robe from ck black endless by calvin klein.",NaN,NaN,"[""Open neckline and front with belt closure at waist"", ""Sleeveless with lace trim at arm openings"", ""Hits at mid-thigh"", ""Silk"", ""Hand wash"", ""Imported"", ""Web ID: 4582655""]","[""xs-s"", ""xs/s"", ""m-l"", ""m/l""]","[""xs/s"", ""m/l""]",black
7611,ck black endless silk lace robe qs5679,$125.00,$125.00,http://www1.macys.com/shop/product/calvin-klein-ck-black-endless-silk-lace-robe-qs5679?id=4582655&categoryid=65739,calvin klein,women - lingerie & shapewear - designer lingerie,macys us,"experience the ultimate luxury with the soft, lightweight silk of this sensational robe from ck black endless by calvin klein.",NaN,NaN,"[""Open neckline and front with belt closure at waist"", ""Sleeveless with lace trim at arm openings"", ""Hits at mid-thigh"", ""Silk"", ""Hand wash"", ""Imported"", ""Web ID: 4582655""]","[""xs-s"", ""xs/s"", ""m-l"", ""m/l""]","[""xs/s"", ""m/l""]",black
11847,ck black endless silk lace robe qs5679,$125.00,$125.00,http://www1.macys.com/shop/product/calvin-klein-ck-black-endless-silk-lace-robe-qs5679?id=4582655&categoryid=65739,calvin klein,women - lingerie & shapewear - designer lingerie,macys us,"experience the ultimate luxury with the soft, lightweight silk of this sensational robe from ck black endless by calvin klein.",NaN,NaN,"[""Open neckline and front with belt closure at waist"", ""Sleeveless with lace trim at arm openings"", ""Hits at mid-thigh"", ""Silk"", ""Hand wash"", ""Imported"", ""Web ID: 4582655""]","[""xs-s"", ""xs/s"", ""m-l"", ""m/l""]","[""xs/s"", ""m/l""]",black
13686,ck black endless silk lace robe qs5679,$125.00,$125.00,http://www1.macys.com/shop/product/calvin-klein-ck-black-endless-silk-lace-robe-qs5679?id=4582655&categoryid=65739,calvin klein,women - lingerie & shapewear - designer lingerie,macys us,"experience the ultimate luxury with the soft, lightweight silk of this sensational robe from ck black endless by calvin klein.",NaN,NaN,"[""Open neckline and front with belt closure at waist"", ""Sleeveless with lace trim at arm openings"", ""Hits at mid-thigh"", ""Silk"", ""Hand wash"", ""Imported"", ""Web ID: 4582655""]","[""xs-s"", ""xs/s"", ""m-l"", ""m/l""]","[""xs/s"", ""m/l""]",black
17840,ck black endless silk lace robe qs5679,$125.00,$125.00,http://www1.macys.com/shop/product/calvin-klein-ck-black-endless-silk-lace-robe-qs5679?id=4582655&categoryid=65739,calvin klein,women - lingerie & shapewear - designer lingerie,macys us,"experience the ultimate luxury with the soft, lightweight silk of this sensational robe from ck black endless by calvin klein.",NaN,NaN,"[""Open neckline and front with belt closure at waist"", ""Sleeveless with lace trim at arm openings"", ""Hits at mid-thigh"", ""Silk"", ""Hand wash"", ""Imported"", ""Web ID: 4582655""]","[""xs-s"", ""xs/s"", ""m-l"", ""m/l""]","[""xs/s"", ""m/l""]",black
1

In [15]:
calvin_klein['product_category'] = calvin_klein['product_category'].apply(lambda x: x.replace("3 for 33 panty essentials", "panties"))

## 1.2 Prices
We expect the prices to be of the form $xx.xx representing dollars and cents. We will find the abnormalities with the regular expression that will give us the type of abnormality and the number of such abnormalities.

In [16]:
import re #regular expression
from collections import Counter 
def find_price_abnorms(data):
    item_arrs = []
    indz_arrs = []
    for ind, item in enumerate(data['price']):
        found = re.search('^\$[0-9]{0,4}[\.]{0,1}[0-9]{0,2}$', item)
        if not found:
            item_arrs.append(item)
            indz_arrs.append(ind)
    print(Counter(item_arrs))
    return indz_arrs

In [17]:
index_ck = find_price_abnorms(calvin_klein)
index_hp = find_price_abnorms(hankypanky)
index_vic = find_price_abnorms(victoria_sec)
index_wac = find_price_abnorms(wacoal)
index_topshop = find_price_abnorms(topshop)

index_ack = find_price_abnorms(amazon_klein)
index_ahp = find_price_abnorms(amazon_panky)
index_avic = find_price_abnorms(amazon_vic_secret)
index_awac = find_price_abnorms(amazon_wacoal)

index_mmk = find_price_abnorms(macys_klein)
index_mhp = find_price_abnorms(macys_panky)
index_mwac = find_price_abnorms(macys_wacoal)

Counter({'$19.99 - $24.99': 12})
Counter()
Counter({'rp474.522': 108, '₡24.769': 102, 'rp543.294': 90, 'rp116.911': 62, '₡25.935': 54, 'rp412.118': 54, 'rp343.857': 36})
Counter()
Counter({'$38.00\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t            \t\t\t\tnow\xa0$12.00': 26, '$38.00\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t            \t\t\t\tnow\xa0$24.00': 25, '$20.00\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t            \t\t\t\tnow\xa0$12.00': 18, '$20.00\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t            \t\t\t\tnow\xa0$6.00': 15, '$35.00\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t       

Abnormalities:
* Calvin Klein has a range price 19.99 - 24.99
* Topshop has a formatinh error: '\xc2\xa0$20.00'
* Victoria's Secret has really weird prices like Rp474.522 and some formating problems: '\xe2\x82\xa124.769'

If we look closer we will find out that formating error are caused by another currency. In Victoria's Secret it is colón (Central American currency). In case of a topshop there are pure formatting problems.

In [18]:
display(victoria_sec[victoria_sec['price']=='₡25.935'].price.head(1))
display(topshop[topshop['price']=='\xa0$12.00'].price.head(1))
display(topshop[topshop['price']=='$38.00\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t            \t\t\t\tnow\xa0$12.00'].price.head(1))

100574    ₡25.935
Name: price, dtype: object

Series([], Name: price, dtype: object)

557    $38.00\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t            \t\t\t\tnow $12.00
Name: price, dtype: object

In [19]:
#display(victoria_sec[victoria_sec['price']=='\xe2\x82\xa125.935'].price.head(1))
#display(topshop[topshop['price']=='\xc2\xa0$10.00'].price.head(1))
#display(topshop[topshop['price']=='\xc2\xa0$45.00\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t            \t\t\t\tnow\xc2\xa0$12.00'].price.head(1))

We will delete badly formatting records from Victoria's Secret. And correct topshop formating problems

In [20]:
victoria_sec.drop(victoria_sec.index[[index_vic]], inplace=True)

In [21]:
topshop['price'] = topshop['price'].apply(lambda x: x.replace("\xc2\xa0", ""))
topshop['mrp'] = topshop['mrp'].apply(lambda x: x.replace("\xc2\xa0", ""))
topshop['price'] = topshop['price'].apply(lambda x: x.replace("\xa0", ""))
topshop['mrp'] = topshop['mrp'].apply(lambda x: x.replace("\xa0", ""))
topshop['price'] = topshop['price'].apply(lambda x: x.replace("\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t            \t\t\t\t", ""))
topshop['mrp'] = topshop['mrp'].apply(lambda x: x.replace("\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t            \t\t\t\t", ""))

In [22]:
index_topshop = find_price_abnorms(topshop)
for ind in index_topshop:
    prices = topshop.loc[ind]['mrp'].split('now')
    topshop.loc[ind, 'mrp'] = prices[0]
    topshop.loc[ind, 'price'] = prices[1]

Counter({'$38.00now$12.00': 26, '$38.00now$24.00': 25, '$20.00now$12.00': 18, '$20.00now$6.00': 15, '$35.00now$20.00': 12, '$15.00now$8.00': 12, '$38.00now$20.00': 12, '$20.00now$10.00': 12, '$30.00now$20.00': 12, '$35.00now$10.00': 10, '$20.00now$15.00': 10, '$35.00now$24.00': 9, '$26.00now$8.00': 3, '$12.00now$4.00': 2, '$10.00now$4.00': 2, '$46.00now$15.00': 2, '$30.00now$8.00': 2, '$45.00now$12.00': 1})


In [23]:
def modify_price(data):    
    data['price'] = data['price'].apply(lambda x: x.replace("$", ""))
    data['mrp'] = data['mrp'].apply(lambda x: x.replace("$", ""))
    data['price'] = data['price'].apply(lambda x: x.replace("19.99 - 24.99", "22.49"))
    data['mrp'] = data['mrp'].apply(lambda x: x.replace("19.99 - 24.99", "22.49"))
    data['price'] = data['price'].apply(lambda x: round(float(x),2))
    data['mrp'] = data['mrp'].apply(lambda x: round(float(x),2))
    return data

In [24]:
brands_old = [calvin_klein,hankypanky,victoria_sec,topshop,wacoal,amazon_klein,amazon_panky,
              amazon_vic_secret,amazon_wacoal,macys_klein,macys_panky,macys_wacoal]
brands = []
for brand in brands_old:
    #brands_price_mod.append(brand)
    brands.append(modify_price(brand))

In [25]:
del brands_old

## 1.3 Delete duplicates part II

Just a small change in description can mislead while recognizing the duplicates. In Victoria's Secret a extra space was inroduced in the description and therefore the records were not recognized as duplicates.


In [26]:
df = brands[2] 
df = df[(df['product_name']=='victoria sport new! lightweight by victoria sport strappy sport bra')&(df['color']=='black')]
df.description.unique()

array([ 'from cycling to circuit training, this medium-support sport bra with a comfy racerback keeps you dry in smooth body-wick fabric. details medium support. pullover style performance & fabric \xa0 perfect for spinning, kickboxing and circuit training nylon performance fabric with 4-way stretch body-wick keeps you dry machine wash. tumble dry. imported polyester/spandex',
       'our #1 sport bra for a reason:\xa0with a supersoft elastic band and wireless cups that breathe easy, this medium-support favorite is ready to bike, box and train hard. details medium support easy, unstructured design adjustable straps wireless; no clasps or hooks strappy back \xa0performance & fabric perfect for spinning, kickboxing and circuit training super-soft elastic band for comfort & movement lightweight and breathable body-wick keeps you cool & dry machine wash. tumble dry. imported polyamide/spandex',
       'our #1 sport bra for a reason: with a supersoft elastic band and wireless cups that brea

In [27]:
brands_name = ['Calvin Klein','Hanky Panky',"Victoria's Secret", 'Topshop', 'Wacoal', 'Amazon Calvin Klein',
               'Amazon Hanky Panky', "Amazon Victoria's Secret", 'Amazon Wacoal',
               'Macys Calvin Klein', 'Macys Hanky Panky', 'Macys Wacoal']

In [28]:
for ind,brand in enumerate(brands):
    df = brand
    #df.index = range(len(df))
    index = df[df['product_name'].str.contains('rolled')].index
    if len(index)>0:
        print(brands_name[ind])
        print(len(index))
    index = df[df['pdp_url'].str.contains('rolled')].index
    if len(index)>0:
        print(brands_name[ind])
        print(len(index))

Hanky Panky
3683
Hanky Panky
3658


In [29]:
df = brands[1]
df.loc[[20,1353]]


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
20,rolled cross-dyed original rise thong,23.0,23.0,http://www.hankypanky.com/collections/rolled-new-cross-dyed-original-rise-thong.html,hankypanky,collections,hankypanky us,scalloped lace and color contrast make this delicate thong a true beauty.,NaN,NaN,"[""One-size thong in our signature stretch lace rolled and ready to go "", "" Fits Sizes 4-14 best (hips measuring 36\""-45\"") "", "" Original Rise fits higher on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" No visible panty line (VPL)"", "" Body: 75% Nylon, 25% Rayon; Trim: 57% Nylon, 30% Rayon, 13% Spandex; Crotch Lining: 100% Supima® Cotton"", "" Made in the USA""]","[""select"", ""one size""]","[""select"", ""one size""]",black/ blue
1353,cross-dyed original rise thong,23.0,23.0,http://www.hankypanky.com/thongs/new-cross-dyed-original-rise-thong.html,hankypanky,thongs,hankypanky us,scalloped lace and color contrast make this delicate thong a true beauty.,NaN,NaN,"[""One-size thong in our signature stretch lace "", "" Fits Sizes 4-14 best (hips measuring 36\""-45\"") "", "" Original Rise fits higher on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" No visible panty line (VPL)"", "" Body: 75% Nylon, 25% Rayon; Trim: 57% Nylon, 30% Rayon, 13% Spandex; Crotch Lining: 100% Supima® Cotton"", "" Made in the USA""]","[""select"", ""one size""]","[""select"", ""one size""]",black/ blue


Topshop missing the color of the item sometimes. The reason is that this product exists only in one specific color and they do not specify it.

In [30]:
for ind, brand in enumerate(brands):
    print(brands_name[ind]+ " "+ str(brand['color'].isnull().sum()))

Calvin Klein 0
Hanky Panky 0
Victoria's Secret 0
Topshop 677
Wacoal 0
Amazon Calvin Klein 0
Amazon Hanky Panky 0
Amazon Victoria's Secret 0
Amazon Wacoal 0
Macys Calvin Klein 0
Macys Hanky Panky 0
Macys Wacoal 0


We will find out how many products have NAs and if there are records of this products with colors as not na.(Two records are the same except for the color, one is na, another is black. That cause for this could be the scrapping method.) The goal is to identify products that will be deleted completely from the dataset and they do not have any representatives with the color specified. In fact, there are only 7 unique products that will be deleted without having representatives with colors.

In [31]:
df = brands[3]
names = df[df['color'].isnull()].product_name.unique()
min_name = []
for name in names:
    num = df[(df['product_name']==name)&(~df['color'].isnull())].shape[0]
    if num == 0:
        min_name.append(name)

min_name

['branded jersey crop top',
 'contrast mesh underwired bra',
 'contrast strappy thong',
 'branded crop top bra',
 'maternity blue basic tankini',
 'maternity blue basic bikini bottoms',
 'strappy mesh bralet']

In [32]:
brands[3] = brands[3].dropna(subset=['color'])

Clean some columns

In [33]:
brands_desc_conc = []
for brand in brands:
    brand['descr_conc'] = brand['description'].apply(lambda x : unicodedata.normalize("NFKD", x))
    brand['descr_conc'] = brand['descr_conc'].apply(lambda x: x.strip())   

    brand['descr_conc'] = brand['descr_conc'].apply(lambda x: x.replace('.', ' '))   
    brand['descr_conc'] = brand['descr_conc'].apply(lambda x: x.replace(',', ' '))
    brand['descr_conc'] = brand['descr_conc'].apply(lambda x: " ".join(x.split()))

    brand['product_name'] = brand['product_name'].apply(lambda x: x.replace('*', ''))
    brand['product_name'] = brand['product_name'].apply(lambda x: x.strip())   
    
    #wacoal has the same product name but sometimes with petite of plus size 
    #brand['product_name'] = brand['product_name'].apply(lambda x: x.replace('plus size', '')) 
    #brand['product_name'] = brand['product_name'].apply(lambda x: x.replace('petite size', '')) 
    
    #some victorias secret products'name were changed, they added "new!" in the middle of the product_name
    brand['product_name'] = brand['product_name'].apply(lambda x: x.replace('new!', ''))
    #sportlace sport lace
    brand['product_name'] = brand['product_name'].apply(lambda x: x.replace('sportlace', 'sport lace'))
    
    brand['product_name'] = brand['product_name'].apply(lambda x: x.replace('trimcheeky', 'trim cheeky'))
    #pink high-leg logo bikini vs pink high leg logo bikini
    brand['product_name'] = brand['product_name'].apply(lambda x: x.replace('rolled ', ''))
    brand['product_name'] = brand['product_name'].apply(lambda x: x.replace('-', ' '))
    brand['product_name'] = brand['product_name'].apply(lambda x: " ".join(x.split())) #to remove double space
    brand['product_name'] = brand['product_name'].apply(lambda x: x.replace('pink bonded strappybralette', 'pink bonded strappy bralette'))


    brand['pdp_url'] = brand['pdp_url'].apply(lambda x: x.replace('rolled-', ''))
    brand['color'] = brand['color'].apply(lambda x: str(x).replace('-', ' '))
    brand['color'] = brand['color'].apply(lambda x: x.replace('/', ' '))

    brand['color'] = brand['color'].apply(lambda x: " ".join(x.split())) #to remove double space

    brands_desc_conc.append(brand)
brands = brands_desc_conc

In [34]:
brands_norm_colors = []
for ind,brand in enumerate(brands):
    df = brand
    #df.index = range(len(df))
    index = df[df['color'].isin(['medium','small','large','one size','one size fits most','null'])].index
    if len(index)>0:
        print(brands_name[ind])
        print(len(index))
    brands_norm_colors.append(df.drop(index))
  
brands = brands_norm_colors

Amazon Calvin Klein
1248
Amazon Hanky Panky
407
Amazon Wacoal
362


In [35]:
brands_norm_products = []
for ind,brand in enumerate(brands):
    df = brand
    #df.index = range(len(df))
    index = df[df['product_name'].str.contains('plus size|petite size')].index
    if len(index)>0:
        print(brands_name[ind])
        print(len(index))
    brands_norm_products.append(df.drop(index))
  
brands = brands_norm_products

Hanky Panky
4661
Macys Hanky Panky
1633


### 1.3.1 Delete real duplicates caused by scraping
We will also mark a duplicate if everthing is the same except for the size (for the sake of simplicity of the first analysis).

In [36]:
def count_duplicates(brand_array, brands_name, set_cols):
    df = pd.DataFrame()
    for ind, brand in enumerate(brand_array):
        dup_df = pd.DataFrame({
            'brand':brands_name[ind], 
            'Number of recs':brand.shape[0], 
            'duplicates':brand.duplicated(subset = set_cols).sum()}, index=[0])
        df = df.append(dup_df)
    return df

set_cols = ['color','descr_conc', 'price', 'mrp', 'product_name', 'product_category']
#set_cols = ['color', 'total_sizes', 'available_size', 'description', 'price', 'mrp', 'product_name', 'product_category']
display(count_duplicates(brands, brands_name, set_cols).transpose())


,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,4747,30344,452880,2405,3518,6124,175,67,3128,15989,6500,14200
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,4113,27889,446489,1951,2831,5224,123,14,2632,14329,5962,12720


In [37]:
def drop_duplicated_records(brand_array, set_cols):
    brands = []
    for brand in brand_array:
        df = brand.drop_duplicates(subset=set_cols,inplace=False)
        #df.index = range(len(df))
        brands.append(df)
    return brands

In [38]:
#brands_sorted = []
#for brand in brands:
#    brands_sorted.append(brand.sort_values(by = ['mrp','price']))
    
set_cols = ['color','descr_conc', 'price', 'mrp', 'product_name', 'product_category']
#set_cols = ['color', 'total_sizes', 'available_size', 'description', 'price', 'mrp', 'product_name', 'product_category']
#set_cols = ['color', 'available_size','description', 'price', 'mrp', 'product_name', 'product_category', 'pdp_url', 'total_sizes']
brands = drop_duplicated_records(brands, set_cols)

### 1.3.2 Find products that have different mrp but everything else is the same
In case something happened while collecting the data, if price was writen in prm or vice a verse we will detect it. We will also take the maximum mrp independent of the size. Some specific size might have a lower price than the others. Or the higher mrp. Therefore, we will sort our dataset first.

In [39]:
set_cols = ['color', 'descr_conc', 'price','product_name', 'product_category']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,634,2455,6391,454,687,900,52,53,496,1660,538,1480
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,0,0,36,0,0,21,0,1,1,0,0,0


In [40]:
#victoria
df = brands[2]
index = df.duplicated(set_cols)
ind = df.index[index][0]
display(df[(df['price']==df.loc[ind].price)&(df['color']==df.loc[ind].color)&(df['product_name']==df.loc[ind].product_name)])
#calvin
df = brands[5]
index = df.duplicated(set_cols)
ind = df.index[index][0]
display(df[(df['price']==df.loc[ind].price)&(df['color']==df.loc[ind].color)&(df['product_name']==df.loc[ind].product_name)])

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
2019,sexy tee demi bra,34.5,14.99,https://www.victoriassecret.com/bras/shop-all-bras/demi-bra-sexy-tee?productid=338253&cataloguetype=ols,victoria's secret,demi bra,victoriassecret us,"sleek and lightweight, this low-cut demi is designed to disappear, showing only your natural shape. wear it day to night under your most curve-hugging styles. lift & lining •lightly lined for shape •underwire cups straps & hooks •front closure or back closure •front-close choices have partially adjustable straps •back-close choices have fully adjustable straps and can be worn classic or cross back •back-close choices have single row of hook-and-eye closures; sizes 32dd, 34d-34dd, 36c-36dd & 38b-38dd have double row of closures details & fabric •infinity edge technology for an invisible look under clothes •imported nylon/spandex",NaN,NaN,NaN,"[""32a"", ""32b"", ""32c"", ""32d"", ""32dd"", ""34a"", ""34b"", ""34c"", ""34d"", ""34dd"", ""36a"", ""36b"", ""36c"", ""36d"", ""36dd"", ""38a"", ""38b"", ""38c"", ""38d"", ""38dd""]",36c,champagne lace,sleek and lightweight this low-cut demi is designed to disappear showing only your natural shape wear it day to night under your most curve-hugging styles lift & lining •lightly lined for shape •underwire cups straps & hooks •front closure or back closure •front-close choices have partially adjustable straps •back-close choices have fully adjustable straps and can be worn classic or cross back •back-close choices have single row of hook-and-eye closures; sizes 32dd 34d-34dd 36c-36dd & 38b-38dd have double row of closures details & fabric •infinity edge technology for an invisible look under clothes •imported nylon/spandex
2152,sexy tee demi bra,36.5,14.99,https://www.victoriassecret.com/bras/shop-all-bras/demi-bra-sexy-tee?productid=338253&cataloguetype=ols,victoria's secret,demi bra,victoriassecret us,"sleek and lightweight, this low-cut demi is designed to disappear, showing only your natural shape. wear it day to night under your most curve-hugging styles. lift & lining •lightly lined for shape •underwire cups straps & hooks •front closure or back closure •front-close choices have partially adjustable straps •back-close choices have fully adjustable straps and can be worn classic or cross back •back-close choices have single row of hook-and-eye closures; sizes 32dd, 34d-34dd, 36c-36dd & 38b-38dd have double row of closures details & fabric •infinity edge technology for an invisible look under clothes •imported nylon/spandex",NaN,NaN,NaN,"[""32a"", ""32b"", ""32c"", ""32d"", ""32dd"", ""34a"", ""34b"", ""34c"", ""34d"", ""34dd"", ""36a"", ""36b"", ""36c"", ""36d"", ""36dd"", ""38a"", ""38b"", ""38c"", ""38d"", ""38dd""]",34dd,champagne lace,sleek and lightweight this low-cut demi is designed to disappear showing only your natural shape wear it day to night under your most curve-hugging styles lift & lining •lightly lined for shape •underwire cups straps & hooks •front closure or back closure •front-close choices have partially adjustable straps •back-close choices have fully adjustable straps and can be worn classic or cross back •back-close choices have single row of hook-and-eye closures; sizes 32dd 34d-34dd 36c-36dd & 38b-38dd have double row of closures details & fabric •infinity edge technology for an invisible look under clothes •imported nylon/spandex


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
273,calvin klein women's everyday calvin full coverage lightly lined bra,38.00,33.63,https://www.amazon.com/-/dp/b01h5nt9ym?th=1&psc=1,calvin-klein,bras,amazon us,this full coverage lightly lined bra provides ultimate comfort featuring new and improved fit offered in an extended size range.,4.5,26,"[ 79% Nylon, 21% Elastane , Imported , Hand Wash , Lightweight fit flexible foam pads with soft cup liner, brushed micro, and plush straps for added comfort , Clean finished double layer wings eliminates exposed stitching against skin , Ballet back and built up wing provides better fit and support , S-hook for convertible straps in conventional and cross-back provides options for every day wear throughout the season ]","32c , 32d , 32dd , 32ddd , 34b , 34c , 34d , 34dd , 34ddd , 36b , 36c , 36d , 36dd , 36ddd , 38b , 38c , 38d , 38dd , 38ddd , 40c , 40d , 40dd","32c , 32d , 32dd , 32ddd , 34b , 34c , 34dd , 34ddd , 36b , 36c , 36d , 36dd , 36ddd , 38b , 38c , 38d , 38dd , 38ddd , 40c , 40d",bare,this full coverage lightly lined bra provides ultimate comfort featuring new and improved fit offered in an extended size range
320,calvin klein women's everyday calvin full coverage lightly lined bra,33.63,33.63,https://www.amazon.com/-/dp/b01h5nt6bi?th=1&psc=1,calvin-klein,bras,amazon us,this full coverage lightly lined bra provides ultimate comfort featuring new and improved fit offered in an extended size range.,4.5,26,"[ 79% Nylon, 21% Elastane , Imported , Hand Wash , Lightweight fit flexible foam pads with soft cup liner, brushed micro, and plush straps for added comfort , Clean finished double layer wings eliminates exposed stitching against skin , Ballet back and built up wing provides better fit and support , S-hook for convertible straps in conventional and cross-back provides options for every day wear throughout the season ]","32c , 32d , 32dd , 32ddd , 34b , 34c , 34d , 34dd , 34ddd , 36b , 36c , 36d , 36dd , 36ddd , 38b , 38c , 38d , 38dd , 38ddd , 40c , 40d , 40dd","32c , 32d , 32dd , 32ddd , 34b , 34d , 34dd , 34ddd , 36b , 36c , 36d , 36dd , 36ddd , 38b , 38c , 38d , 38dd , 38ddd , 40c , 40d",bare,this full coverage lightly lined bra provides ultimate comfort featuring new and improved fit offered in an extended size range


In [41]:
brands_sorted = []
for brand in brands:
    brands_sorted.append(brand.sort_values(by = 'mrp', ascending=False))
brands = drop_duplicated_records(brands_sorted, set_cols)

### 1.3.3 Find products that have different price but everything else is the same
Lets say they made a discount between the two scap sessions, so we get different price. Some specific size might have a lower price than the others. Or the higher mrp. Therefore, we will sort our dataset first.

In [42]:
set_cols = ['color', 'descr_conc', 'mrp','product_name', 'product_category']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,634,2455,6355,454,687,879,52,52,495,1660,538,1480
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,55,17,1106,5,71,617,16,0,385,315,0,297


In [43]:
#victoria
df = brands[2]
index = df.duplicated(set_cols)
ind = df.index[index][0]
display(df[(df['mrp']==df.loc[ind].mrp)&(df['color']==df.loc[ind].color)&(df['product_name']==df.loc[ind].product_name)])
#calvin
#df = brands_price_mod[5]
#index = df.duplicated(set_cols)
#ind = df.index[index][0]
#display(df[(df['price']==df.loc[ind].price)&(df['color']==df.loc[ind].color)&(df['product_name']==df.loc[ind].product_name)])

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
285431,very sexy lace trim babydoll,68.0,39.99,https://www.victoriassecret.com/lingerie/shop-all-lingerie/lace-trim-babydoll-very-sexy?productid=317741&cataloguetype=ols,victoria's secret,lace-trim babydoll,victoriassecret us,"this sexy babydoll is all show, with push-up padding, a flyaway back and a matching v-string. push-up padding with underwire cups elastic under bust back hook-and-eye closures 28¼"" from shoulder; hits at upper thigh lace cups sheer skirt with flyaway back matching v-string panty imported nylon/spandex/polyester",NaN,NaN,NaN,"[""32b"", ""32c"", ""32d"", ""34b"", ""34c"", ""34d"", ""36b"", ""36c"", ""36d"", ""38b"", ""38c"", ""38d""]",36c,black,"this sexy babydoll is all show with push-up padding a flyaway back and a matching v-string push-up padding with underwire cups elastic under bust back hook-and-eye closures 281⁄4"" from shoulder; hits at upper thigh lace cups sheer skirt with flyaway back matching v-string panty imported nylon/spandex/polyester"
359585,very sexy lace trim babydoll,68.0,68.00,https://www.victoriassecret.com/lingerie/shop-all-lingerie/lace-trim-babydoll-very-sexy?productid=317741&cataloguetype=ols,victoria's secret,lace-trim babydoll,victoriassecret us,"this sexy babydoll is all show, with push-up padding, a flyaway back and a matching v-string. push-up padding with underwire cups elastic under bust back hook-and-eye closures 28¼"" from shoulder; hits at upper thigh lace cups sheer skirt with flyaway back matching v-string panty imported nylon/spandex/polyester",NaN,NaN,NaN,"[""32b"", ""32c"", ""32d"", ""34b"", ""34c"", ""34d"", ""36b"", ""36c"", ""36d"", ""38b"", ""38c"", ""38d""]",36c,black,"this sexy babydoll is all show with push-up padding a flyaway back and a matching v-string push-up padding with underwire cups elastic under bust back hook-and-eye closures 281⁄4"" from shoulder; hits at upper thigh lace cups sheer skirt with flyaway back matching v-string panty imported nylon/spandex/polyester"


In [44]:
brands_sorted = []
for brand in brands:
    #sort so that the discounted prices will not be removed
    brands_sorted.append(brand.sort_values(by = 'price'))

In [45]:
brands = drop_duplicated_records(brands_sorted, set_cols)

### 1.3.4 Find products that have different price and mrp but everything else is the same
That might look as a scapping issue as well. I think it would be better to delete both of such records from the dataset as we do not know what is the truth (in fact we will never now it ;))

In [46]:
set_cols = ['color', 'descr_conc', 'pdp_url', 'product_name','product_category']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,579,2438,5249,449,616,262,36,52,110,1345,538,1183
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,0,0,711,0,0,33,8,5,5,38,0,0


In [47]:
df = brands[5]
index = df.duplicated(set_cols)
ind = df.index[index][2]
display(df[(df['pdp_url']==df.loc[ind].pdp_url)&(df['color']==df.loc[ind].color)&(df['product_name']==df.loc[ind].product_name)])


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
18114,calvin klein women's modern cotton boyshort panty,7.99,7.99,https://www.amazon.com/-/dp/b01kubisb6?th=1&psc=1,calvin-klein,panties,amazon us,a calvin klein modern cotton boy short in soft cotton for breathable comfort and a comfortable waistband featuring the iconic calvin klein logo.,4.4,399,"[ 53% Cotton, 35% Modal, 12% Elastane , Imported , Machine Wash , Repeating Calvin Klein logo waistband , Cotton gusset , Soft and comfortable cotton boyshort , Tag less , Soft, stretch cotton ]","x-small , small , medium , large , x-large","x-small , medium , large",chevron outline logo grey heather,a calvin klein modern cotton boy short in soft cotton for breathable comfort and a comfortable waistband featuring the iconic calvin klein logo
4637,calvin klein women's modern cotton boyshort panty,11.55,11.55,https://www.amazon.com/-/dp/b01kubisb6?th=1&psc=1,calvin-klein,panties,amazon us,a calvin klein modern cotton boy short in soft cotton for breathable comfort and a comfortable waistband featuring the iconic calvin klein logo.,4.4,403,"[ 53% Cotton, 35% Modal, 12% Elastane , Imported , Machine Wash , Repeating Calvin Klein logo waistband , Cotton gusset , Soft and comfortable cotton boyshort , Tag less , Soft, stretch cotton ]","x-small , small , medium , large , x-large","medium , large",chevron outline logo grey heather,a calvin klein modern cotton boy short in soft cotton for breathable comfort and a comfortable waistband featuring the iconic calvin klein logo


In [48]:
df = brands[2]
index = df.duplicated(set_cols)
ind = df.index[index][2]
display(df[(df['pdp_url']==df.loc[ind].pdp_url)&(df['color']==df.loc[ind].color)&(df['product_name']==df.loc[ind].product_name)])
#calvin

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
27181,cotton lingerie bikini panty,8.5,8.5,https://www.victoriassecret.com/panties/shop-all-panties/bikini-panty-cotton-lingerie?productid=313566&cataloguetype=ols,victoria's secret,bikini panty,victoriassecret us,"our most-loved everyday panty is naturally sexy and comfy, in silky-soft and stretchy cotton. in lots of colors and pretty prints, our cotton undies ensure your lingerie drawer is anything but ordinary. picot trim mid rise moderate back coverage imported cotton/lycra® spandex; pure cotton panel",4.4,311.0,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",s,white,our most-loved everyday panty is naturally sexy and comfy in silky-soft and stretchy cotton in lots of colors and pretty prints our cotton undies ensure your lingerie drawer is anything but ordinary picot trim mid rise moderate back coverage imported cotton/lycra® spandex; pure cotton panel
378788,cotton lingerie bikini panty,10.5,10.5,https://www.victoriassecret.com/panties/shop-all-panties/bikini-panty-cotton-lingerie?productid=313566&cataloguetype=ols,victoria's secret,bikini panty,victoriassecret us,"our most-loved everyday panty is naturally sexy and comfy, in silky-soft and stretchy cotton. in lots of colors and pretty prints, our cotton undies ensure your lingerie drawer is anything but ordinary. picot trim mid rise moderate back coverage imported cotton/lycra® spandex; pure cotton panel",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",s,white,our most-loved everyday panty is naturally sexy and comfy in silky-soft and stretchy cotton in lots of colors and pretty prints our cotton undies ensure your lingerie drawer is anything but ordinary picot trim mid rise moderate back coverage imported cotton/lycra® spandex; pure cotton panel


In [49]:
brands[2] = brands[2].drop_duplicates(set_cols, keep = False)
brands[5] = brands[5].drop_duplicates(set_cols, keep = False)
brands[6] = brands[6].drop_duplicates(set_cols, keep = False)
brands[7] = brands[7].drop_duplicates(set_cols, keep = False)
brands[8] = brands[8].drop_duplicates(set_cols, keep = False)
brands[9] = brands[9].drop_duplicates(set_cols, keep = False)

### 1.3.5 Find products that have different description but everything else is the same
In case during the scaping part the description was changed a bit but everything else remained the same, even url. Example below.

In [50]:
set_cols = ['color','price', 'mrp','product_name', 'product_category', 'pdp_url']
display(count_duplicates(brands, brands_name, set_cols).transpose())


,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,579,2438,3835,449,616,204,22,42,101,1269,538,1183
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,0,74,18,27,8,0,0,0,0,0,2,4


In [51]:
df = brands[2]
index = df.duplicated(set_cols)
ind = df.index[index][0]
display(df[(df['product_category']==df.loc[ind].product_category)&(df['product_name']==df.loc[ind].product_name)&(df['pdp_url']==df.loc[ind].pdp_url)&(df['mrp']==df.loc[ind].mrp)&(df['color']==df.loc[ind].color)&(df['price']==df.loc[ind].price)][['pdp_url','product_name','description']])

,pdp_url,product_name,description
96811,https://www.victoriassecret.com/panties/shop-all-panties/ruched-back-hiphugger-panty-cotton-lingerie?productid=336099&cataloguetype=ols,cotton lingerie ruched back hiphugger panty,"this comfy hiphugger is naturally sexy in silky-soft, stretchy cotton, with flirty back ruching.in lots of colors and pretty prints, our cotton undies ensure your lingerie drawer is anything but ordinary. mid rise moderate back coverage imported cotton/lycra® spandex; pure cotton panel"
157466,https://www.victoriassecret.com/panties/shop-all-panties/ruched-back-hiphugger-panty-cotton-lingerie?productid=336099&cataloguetype=ols,cotton lingerie ruched back hiphugger panty,"this comfy hiphugger is naturally sexy in silky-soft, stretchy cotton, with flirty back ruching.in lots of colors and pretty prints, our cotton undies ensure your lingerie drawer is anything but ordinary. picot trim mid rise moderate back coverage imported cotton/lycra® spandex; pure cotton panel"


In [52]:
brands = drop_duplicated_records(brands, set_cols)

### 1.3.6 Find records where everything is the same appart from product_category
In some brands if we change the prouct_category the url will also be changed, so we will not take url into consideration.

In [53]:
df = brands[4]
df[(df['color']=='denim')&(df['product_name']=="b.tempt'd lace kiss thong")][set_cols +['product_category']]

,color,price,mrp,product_name,product_category,pdp_url,product_category
417,denim,7.8,13.0,b.tempt'd lace kiss thong,collections,http://btemptd.wacoal-america.com/b-tempt-d-lace-kiss-thong-btemptd-970182-p.html,collections
938,denim,7.8,13.0,b.tempt'd lace kiss thong,panties,http://btemptd.wacoal-america.com/b-tempt-d-lace-kiss-thong-btemptd-970182-p.html,panties


In [54]:
df = brands[9]
df[(df['color']=='riley')&(df['product_name']=='seductive comfort push up add a size bra qf1446')][set_cols + ['pdp_url','product_category']]

,color,price,mrp,product_name,product_category,pdp_url,pdp_url,product_category
5145,riley,48.0,48.0,seductive comfort push up add a size bra qf1446,women - lingerie & shapewear - bras,http://www1.macys.com/shop/product/calvin-klein-seductive-comfort-push-up-add-a-size-bra-qf1446?id=2891332&categoryid=55799,http://www1.macys.com/shop/product/calvin-klein-seductive-comfort-push-up-add-a-size-bra-qf1446?id=2891332&categoryid=55799,women - lingerie & shapewear - bras
3932,riley,48.0,48.0,seductive comfort push up add a size bra qf1446,women - lingerie & shapewear - designer lingerie,http://www1.macys.com/shop/product/calvin-klein-seductive-comfort-push-up-add-a-size-bra-qf1446?id=2891332&categoryid=65739,http://www1.macys.com/shop/product/calvin-klein-seductive-comfort-push-up-add-a-size-bra-qf1446?id=2891332&categoryid=65739,women - lingerie & shapewear - designer lingerie


In [55]:
set_cols = ['color', 'price', 'mrp','product_name', 'descr_conc']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,579,2364,3817,422,608,204,22,42,101,1269,536,1179
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,71,1142,31,0,224,0,0,0,0,592,253,580


In [56]:
brands = drop_duplicated_records(brands, set_cols)

### 1.3.6 Find products that have different product_name but everything else is the same

In [57]:
set_cols = ['color', 'price', 'mrp', 'descr_conc', 'pdp_url']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,508,1222,3786,422,384,204,22,42,101,677,283,599
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,0,0,2,0,1,3,0,0,0,0,0,0


In [58]:
#victoria
df = brands[2]
index = df.duplicated(set_cols)
ind = df.index[index][0]
display(df[(df['pdp_url']==df.loc[ind].pdp_url)&(df['mrp']==df.loc[ind].mrp)&(df['color']==df.loc[ind].color)&(df['price']==df.loc[ind].price)][['pdp_url','product_name']])

,pdp_url,product_name
370563,https://www.victoriassecret.com/pink/apparel-activewear/super-softmuscle-tank-pink?productid=330375&cataloguetype=ols,pink super softmuscle tank
242950,https://www.victoriassecret.com/pink/apparel-activewear/super-softmuscle-tank-pink?productid=330375&cataloguetype=ols,pink super soft muscle tank


In [59]:
#calvin
df = brands[5]
index = df.duplicated(set_cols)
ind = df.index[index][0]
display(df[(df['pdp_url']==df.loc[ind].pdp_url)&(df['price']==df.loc[ind].price)&(df['mrp']==df.loc[ind].mrp)&(df['color']==df.loc[ind].color)&(df['price']==df.loc[ind].price)][['pdp_url','product_name']])

,pdp_url,product_name
8561,https://www.amazon.com/-/dp/b01nc3118b?th=1&psc=1,calvin klein women's ck one cotton unlined bralette
12681,https://www.amazon.com/-/dp/b01nc3118b?th=1&psc=1,calvin klein women's standard ck one cotton unlined bralette


In [60]:
brands = drop_duplicated_records(brands, set_cols)

In [61]:
set_cols = ['color', 'price', 'mrp', 'descr_conc']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,508,1222,3784,422,383,201,22,42,101,677,283,599
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,5,132,42,0,0,1,0,0,0,0,0,0


In [62]:
#calvin
#the same short description
df = brands[0]
index = df.duplicated(set_cols)
ind = df.index[index][2]
display(df[(df['descr_conc']==df.loc[ind].descr_conc)&(df['mrp']==df.loc[ind].mrp)&(df['color']==df.loc[ind].color)&(df['price']==df.loc[ind].price)][['pdp_url','product_name','description']])

,pdp_url,product_name,description
132,http://www.calvinklein.us/en/womens-clothing/women-three-for-panties/cheeky-bikini-with-lace-51563537-624,cheeky bikini with lace,silky fabric with a lace trim and moderate rear coverage.
45,http://www.calvinklein.us/en/womens-clothing/women-panties/cheeky-hipster-with-lace-51563538-265,cheeky hipster with lace,silky fabric with a lace trim and moderate rear coverage.


In [63]:
#hanky panky
#the same description for a bit different products
df = brands[1]
index = df.duplicated(set_cols)
index = df.index[index]
for ind in index:
    display(df[(df['descr_conc']==df.loc[ind].descr_conc)&(df['mrp']==df.loc[ind].mrp)&(df['color']==df.loc[ind].color)&(df['price']==df.loc[ind].price)][['pdp_url','product_name','description']])

,pdp_url,product_name,description
11345,http://www.hankypanky.com/collections/breton-stripe-original-rise-thong.html,breton stripe original rise thong,blue and white feels oh so right.
12576,http://www.hankypanky.com/collections/breton-stripe-low-rise-thong.html,breton stripe low rise thong,blue and white feels oh so right.


,pdp_url,product_name,description
20847,http://www.hankypanky.com/thongs/breton-stripe-low-rise-thong.html,breton stripe low rise thong,blue and white feels oh so right.
20864,http://www.hankypanky.com/thongs/breton-stripe-original-rise-thong.html,breton stripe original rise thong,blue and white feels oh so right.


,pdp_url,product_name,description
566,http://www.hankypanky.com/thongs/signature-lace-low-rise-thong.html,signature lace low rise thong,a must-have for every woman's lingerie drawer.
429,http://www.hankypanky.com/thongs/signature-lace-original-rise-thong.html,signature lace original rise thong,a must-have for every woman's lingerie drawer.


,pdp_url,product_name,description
421,http://www.hankypanky.com/thongs/signature-lace-original-rise-thong.html,signature lace original rise thong,a must-have for every woman's lingerie drawer.
558,http://www.hankypanky.com/thongs/signature-lace-low-rise-thong.html,signature lace low rise thong,a must-have for every woman's lingerie drawer.


,pdp_url,product_name,description
419,http://www.hankypanky.com/thongs/signature-lace-original-rise-thong.html,signature lace original rise thong,a must-have for every woman's lingerie drawer.
556,http://www.hankypanky.com/thongs/signature-lace-low-rise-thong.html,signature lace low rise thong,a must-have for every woman's lingerie drawer.


...

,pdp_url,product_name,description
34704,http://www.hankypanky.com/thongs/5-pack-bare-godiva-hi-rise-thongs.html,"5 pack of bare ""godiva"" hi rise thongs",truly-- what to wear when you want to look bare®.
1743,http://www.hankypanky.com/thongs/5-pack-bare-eve-natural-rise-thongs.html,"5 pack of bare ""eve"" natural rise thongs",truly-- what to wear when you want to look bare®.


,pdp_url,product_name,description
2829,http://www.hankypanky.com/collections/5-pack-original-rise-thongs-baiscs.html,5 pack original rise thongs,your favorite basic colored thongs - packaged together.
2470,http://www.hankypanky.com/collections/5-pack-low-rise-thongs-basic.html,5 pack low rise thongs,your favorite basic colored thongs - packaged together.


,pdp_url,product_name,description
936,http://www.hankypanky.com/lingerie/heather-jersey-cotton-sleep-set.html,40th anniversary cotton sleep set,"it all started as a gift…in 1977 designer gale epstein created and bra and panty set out of hand-embroidered handkerchiefs as a birthday gift for her friend, lida orzeck. inspired by this ingenious concept, the friends became business partners and named their new company after the original design, hanky panky."
755,http://www.hankypanky.com/lingerie/heather-jersey-chemise-1.html,40th anniversary cotton chemise,"it all started as a gift…in 1977 designer gale epstein created and bra and panty set out of hand-embroidered handkerchiefs as a birthday gift for her friend, lida orzeck. inspired by this ingenious concept, the friends became business partners and named their new company after the original design, hanky panky."


,pdp_url,product_name,description
2511,http://www.hankypanky.com/thongs/original-rise-red-thong-rose-bouquet-of-12.html,original rise thong rose bouquet of 12,"give the gift that lasts as long as your love for her. 12 long stem roses, in a floral box with tissue and a packet of our exclusive lingerie wash."
2020,http://www.hankypanky.com/thongs/low-rise-red-thong-rose-bouquet-of-12.html,low rise thong rose bouquet of 12,"give the gift that lasts as long as your love for her. 12 long stem roses, in a floral box with tissue and a packet of our exclusive lingerie wash."


,pdp_url,product_name,description
2510,http://www.hankypanky.com/thongs/original-rise-red-thong-rose-bouquet-of-12.html,original rise thong rose bouquet of 12,"give the gift that lasts as long as your love for her. 12 long stem roses, in a floral box with tissue and a packet of our exclusive lingerie wash."
2019,http://www.hankypanky.com/thongs/low-rise-red-thong-rose-bouquet-of-12.html,low rise thong rose bouquet of 12,"give the gift that lasts as long as your love for her. 12 long stem roses, in a floral box with tissue and a packet of our exclusive lingerie wash."


In [64]:
#vicotiras secret
#some products have the same ids in the pdp_url but have a bit different product_name. 
#the links do not work anymore, therefore we will assume that due to the same id and also due to almost the same names
#that are the same products
df = brands[2]
index = df.duplicated(set_cols)
index = df.index[index]
del_index = []
for ind in index:
    g = df[(df['descr_conc']==df.loc[ind].descr_conc)&(df['price']==df.loc[ind].price)&(df['mrp']==df.loc[ind].mrp)&(df['color']==df.loc[ind].color)&(df['price']==df.loc[ind].price)]
    display(g[['pdp_url','product_name','description']])
    item = 'very sexy fishnet lace kimono|very sexy high neck lace|pink date|pink wear|allover lace|old school logo|racerback|victoria sport seamless|pink lace|body by victoria lace|pink embroidered'
    tmp_ind = g[g['product_name'].str.contains(item)].index.tolist()
    del_index.append(tmp_ind)

,pdp_url,product_name,description
20268,https://www.victoriassecret.com/panties/shop-all-panties/dotted-mesh-bikini-panty-allover-lace-from-cotton-lingerie?productid=227301&cataloguetype=ols,allover lace from cotton lingerie dotted mesh bikini panty,dotted mesh in the ultimate everyday shape makes this pretty panty your go-to underneath anything. supersoft & stretchy elastic at waist & legs lace trim at legs mid rise moderate back coverage imported nylon
323727,https://www.victoriassecret.com/panties/shop-all-panties/dotted-mesh-bikini-panty-cotton-lingerie?productid=227301&cataloguetype=ols,cotton lingerie dotted mesh bikini panty,dotted mesh in the ultimate everyday shape makes this pretty panty your go-to underneath anything. supersoft & stretchy elastic at waist & legs lace trim at legs mid rise moderate back coverage imported nylon


,pdp_url,product_name,description
20269,https://www.victoriassecret.com/panties/shop-all-panties/dotted-mesh-bikini-panty-allover-lace-from-cotton-lingerie?productid=227301&cataloguetype=ols,allover lace from cotton lingerie dotted mesh bikini panty,dotted mesh in the ultimate everyday shape makes this pretty panty your go-to underneath anything. supersoft & stretchy elastic at waist & legs lace trim at legs mid rise moderate back coverage imported nylon
323728,https://www.victoriassecret.com/panties/shop-all-panties/dotted-mesh-bikini-panty-cotton-lingerie?productid=227301&cataloguetype=ols,cotton lingerie dotted mesh bikini panty,dotted mesh in the ultimate everyday shape makes this pretty panty your go-to underneath anything. supersoft & stretchy elastic at waist & legs lace trim at legs mid rise moderate back coverage imported nylon


,pdp_url,product_name,description
27307,https://www.victoriassecret.com/panties/shop-all-panties/dotted-mesh-thong-panty-allover-lace-from-cotton-lingerie?productid=227307&cataloguetype=ols,allover lace from cotton lingerie dotted mesh thong panty,dotted mesh in a sexy little shape makes this pretty thong perfectly flirty under anything. supersoft & stretchy elastic at waist & legs lace trim at legs mid rise no back coverage imported nylon
348352,https://www.victoriassecret.com/panties/shop-all-panties/dotted-mesh-thong-panty-cotton-lingerie?productid=227307&cataloguetype=ols,cotton lingerie dotted mesh thong panty,dotted mesh in a sexy little shape makes this pretty thong perfectly flirty under anything. supersoft & stretchy elastic at waist & legs lace trim at legs mid rise no back coverage imported nylon


,pdp_url,product_name,description
354885,https://www.victoriassecret.com/panties/shop-all-panties/old-school-logo-waist-shortie-panty-cotton-lingerie?productid=334153&cataloguetype=ols,cotton lingerie old school logo waist shortie panty,"the shortie gets an update with an on-trend sheer logo waist, in silky-soft and stretchy cotton that’s naturally sexy. in lots of colors and pretty prints, our cotton undies ensure your lingerie drawer is anything but ordinary. sheer elastic waistband with logo mid rise full back coverage: shows curves, not skin imported cotton/nylon/spandex"
31668,https://www.victoriassecret.com/panties/shop-all-panties/logo-waist-shortie-panty-cotton-lingerie?productid=330277&cataloguetype=ols,cotton lingerie logo waist shortie panty,"the shortie gets an update with an on-trend sheer logo waist, in silky-soft and stretchy cotton that’s naturally sexy. in lots of colors and pretty prints, our cotton undies ensure your lingerie drawer is anything but ordinary. sheer elastic waistband with logo mid rise full back coverage: shows curves, not skin imported cotton/nylon/spandex"


,pdp_url,product_name,description
354886,https://www.victoriassecret.com/panties/shop-all-panties/old-school-logo-waist-shortie-panty-cotton-lingerie?productid=334153&cataloguetype=ols,cotton lingerie old school logo waist shortie panty,"the shortie gets an update with an on-trend sheer logo waist, in silky-soft and stretchy cotton that’s naturally sexy. in lots of colors and pretty prints, our cotton undies ensure your lingerie drawer is anything but ordinary. sheer elastic waistband with logo mid rise full back coverage: shows curves, not skin imported cotton/nylon/spandex"
31661,https://www.victoriassecret.com/panties/shop-all-panties/logo-waist-shortie-panty-cotton-lingerie?productid=330277&cataloguetype=ols,cotton lingerie logo waist shortie panty,"the shortie gets an update with an on-trend sheer logo waist, in silky-soft and stretchy cotton that’s naturally sexy. in lots of colors and pretty prints, our cotton undies ensure your lingerie drawer is anything but ordinary. sheer elastic waistband with logo mid rise full back coverage: shows curves, not skin imported cotton/nylon/spandex"


...

,pdp_url,product_name,description
283037,https://www.victoriassecret.com/pink/all-bras/date-push-up-bralette?productid=338303&cataloguetype=ols,date push up bralette,"meet summer’s must-have bralette, featuring the prettiest lace plus a new v-wire that hides under the lowest cut tops. push-up padding plus a lacy racerback. push-up padding hook-and-eye closure adjustable straps v-wire racerback imported nylon/spandex"
8417,https://www.victoriassecret.com/pink/panties/date-push-up-bralette-pink?productid=335644&cataloguetype=ols,pink date push up bralette,"meet summer’s must-have bralette, featuring the prettiest lace plus a new v-wire that hides under the lowest cut tops. push-up padding plus a lacy racerback. push-up padding hook-and-eye closure adjustable straps v-wire racerback imported nylon/spandex"


,pdp_url,product_name,description
8410,https://www.victoriassecret.com/pink/panties/date-push-up-bralette-pink?productid=335644&cataloguetype=ols,pink date push up bralette,"meet summer’s must-have bralette, featuring the prettiest lace plus a new v-wire that hides under the lowest cut tops. push-up padding plus a lacy racerback. push-up padding hook-and-eye closure adjustable straps v-wire racerback imported nylon/spandex"
283030,https://www.victoriassecret.com/pink/all-bras/date-push-up-bralette?productid=338303&cataloguetype=ols,date push up bralette,"meet summer’s must-have bralette, featuring the prettiest lace plus a new v-wire that hides under the lowest cut tops. push-up padding plus a lacy racerback. push-up padding hook-and-eye closure adjustable straps v-wire racerback imported nylon/spandex"


,pdp_url,product_name,description
8409,https://www.victoriassecret.com/pink/panties/date-push-up-bralette-pink?productid=335644&cataloguetype=ols,pink date push up bralette,"meet summer’s must-have bralette, featuring the prettiest lace plus a new v-wire that hides under the lowest cut tops. push-up padding plus a lacy racerback. push-up padding hook-and-eye closure adjustable straps v-wire racerback imported nylon/spandex"
283029,https://www.victoriassecret.com/pink/all-bras/date-push-up-bralette?productid=338303&cataloguetype=ols,date push up bralette,"meet summer’s must-have bralette, featuring the prettiest lace plus a new v-wire that hides under the lowest cut tops. push-up padding plus a lacy racerback. push-up padding hook-and-eye closure adjustable straps v-wire racerback imported nylon/spandex"


,pdp_url,product_name,description
39272,https://www.victoriassecret.com/lingerie/shop-all-lingerie/high-neck-lace-up-romper-very-sexy?productid=334099&cataloguetype=ols,very sexy high neck lace up romper,"go all out in a sheer romper with a modern high-neck, a lace-up front and a sexy open back. sheer, unlined adjustable hook closure at neck high neck with lace-up front, elastic waistband and open back 1 1/8"" inseam imported nylon/spandex/polyester"
124100,https://www.victoriassecret.com/lingerie/shop-all-lingerie/high-neck-lace-up-romper-dream-angels?productid=334099&cataloguetype=ols,dream angels high neck lace up romper,"go all out in a sheer romper with a modern high-neck, a lace-up front and a sexy open back. sheer, unlined adjustable hook closure at neck high neck with lace-up front, elastic waistband and open back 1 1/8"" inseam imported nylon/spandex/polyester"


,pdp_url,product_name,description
578,https://www.victoriassecret.com/lingerie/shop-all-lingerie/fishnet-lace-kimono-very-sexy?productid=331793&cataloguetype=ols,very sexy fishnet lace kimono,"the ultimate lacy layer, with a plunge front and open back. allover lace ties at back neck and center front flutter sleeves 38¼'' from waist; high-low hem hits at knee imported nylon/polyester"
241355,https://www.victoriassecret.com/lingerie/shop-all-lingerie/fishnet-lace-dress-very-sexy?productid=331793&cataloguetype=ols,very sexy fishnet lace dress,"the ultimate lacy layer, with a plunge front and open back. allover lace ties at back neck and center front flutter sleeves 38¼'' from waist; high-low hem hits at knee imported nylon/polyester"


In [65]:
#del_index = [793,794,923,1473,1474,924,2163,2165,2164,2166,2555,2557,2657,2643,2646,2725,2726,2727,2781,2782,2783,2739,2749,2787,2745,2784,2750,2754,2940,2942,2943,3025,3026,3300,3396,3397,3398,3399,
#3467,3470,3764,3782]
len(del_index)

42

In [66]:
del_index = [item for sublist in del_index for item in sublist]

In [67]:
brands[2].drop(del_index,inplace = True)

In [68]:
#amazon calbin klein
df = brands[5]
index = df.duplicated(set_cols)
index = df.index[index]
for ind in index:
    display(df[(df['descr_conc']==df.loc[ind].descr_conc)&(df['price']==df.loc[ind].price)&(df['mrp']==df.loc[ind].mrp)&(df['color']==df.loc[ind].color)&(df['price']==df.loc[ind].price)][['pdp_url','product_name','description','color']])

,pdp_url,product_name,description,color
12067,https://www.amazon.com/-/dp/b01n3ckijz?th=1&psc=1,calvin klein women's standard ck one cotton unlined bralette,a soft jersey calvin klein underwear bralette styled with a contrast logo waistband. adjustable shoulder straps. double-layered.,dessert sunset
9082,https://www.amazon.com/-/dp/b01nb1twio?th=1&psc=1,calvin klein women's ck one cotton unlined bralette,a soft jersey calvin klein underwear bralette styled with a contrast logo waistband. adjustable shoulder straps. double-layered.,dessert sunset


Amazon has the same product but listed in two different pages with some of the colors having different colors. We will deal with this at the very end.

### 1.3.7 Find product that have different product categories and different price

In [69]:
set_cols = ['color', 'descr_conc', 'pdp_url', 'mrp', 'product_name']
display(count_duplicates(brands, brands_name, set_cols).transpose())
df = brands[4]
index = df.duplicated(set_cols)
ind = df.index[index][0]
display(df[(df['color']==df.loc[ind]['color'])&(df['description']==df.loc[ind]['description'])&(df['pdp_url']==df.loc[ind]['pdp_url'])])

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,508,1222,3742,422,383,201,22,42,101,677,283,599
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,0,0,0,0,3,0,0,0,0,0,0,0


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
1065,b.sultry bikini,20.0,12.0,http://btemptd.wacoal-america.com/b-sultry-bikini-btemptd-943261-p.html,wacoal,panties,btemptd us,"a little lace and a lot of style put our b.sultry bikini in the mix for perfect panties. wear this bikini with the matching bra, chemise, or garter belt and be sultry without even trying.",NaN,NaN,"[Chantilly lace front,Stretch mesh body,100% cotton lining,Matching bras 951261, 953261, 959261 (sold separately), Body: 88% Nylon/12% Spandex; Panel Lining: 100% cotton ,Hand wash warm,Wash colors separately,Use non-chlorine bleach only when needed,Hang to dry,Do not iron,Do not dry clean]","s,m,l,xl","l,m,s,xl",bridal white,a little lace and a lot of style put our b sultry bikini in the mix for perfect panties wear this bikini with the matching bra chemise or garter belt and be sultry without even trying
2217,b.sultry bikini,20.0,14.0,http://btemptd.wacoal-america.com/b-sultry-bikini-btemptd-943261-p.html,wacoal,collections,btemptd us,"a little lace and a lot of style put our b.sultry bikini in the mix for perfect panties. wear this bikini with the matching bra, chemise, or garter belt and be sultry without even trying.",NaN,NaN,"[Chantilly lace front,Stretch mesh body,100% cotton lining,Matching bras 951261, 953261, 959261 (sold separately), Body: 88% Nylon/12% Spandex; Panel Lining: 100% cotton ,Hand wash warm,Wash colors separately,Use non-chlorine bleach only when needed,Hang to dry,Do not iron,Do not dry clean]","s,m,l,xl","l,m,s,xl",bridal white,a little lace and a lot of style put our b sultry bikini in the mix for perfect panties wear this bikini with the matching bra chemise or garter belt and be sultry without even trying


In [70]:
brands_sorted = []
for brand in brands:
    #so that the more discounted item stays
    brands_sorted.append(brand.sort_values(by='price'))
brands = drop_duplicated_records(brands_sorted, set_cols)

In [71]:
#set_cols = ['color', 'descr_conc', 'pdp_url', 'price', 'product_name']
#display(count_duplicates(brands, brands_name, set_cols).transpose())

### 1.3.8 Find products that have the same description but different prices, mrp and product name
Also the same pdp_url

In [72]:
brands[0][brands[0].pdp_url=='http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-ck-id-triangle-black']

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
4537,ck id thin waistband string bikini,20.0,20.0,http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-ck-id-triangle-black,calvin klein,panty sets,calvin klein us,"a ck id unlined triangle bra and string bikini set featuring a convertible design, cotton stretch fabric and a thin logo band revisiting cotton classic styling and exploring a new shape.",NaN,NaN,"[""ck id thin waistband string bikini"",""bikini silhouette"",""thin logo band"",""full gusset"",""rear coverage"",""machine wash"",""95% cotton 5% spandex"",""imported"",""Style Number 51471754"",""universal style Number QF1754""]","xs,s,m,l,xl","xs,s,m,l,xl",black,a ck id unlined triangle bra and string bikini set featuring a convertible design cotton stretch fabric and a thin logo band revisiting cotton classic styling and exploring a new shape
4536,ck id thin waistband unlined triangle bra,32.0,32.0,http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-ck-id-triangle-black,calvin klein,panty sets,calvin klein us,"a ck id unlined triangle bra and string bikini set featuring a convertible design, cotton stretch fabric and a thin logo band revisiting cotton classic styling and exploring a new shape.",NaN,NaN,"[""ck id thin waistband unlined triangle bra"",""convertible conventional, crossback + halter design"",""triangle design + straps"",""front closure"",""thin logo band"",""unlined"",""machine wash"",""95% cotton 5% spandex"",""imported"",""Style Number 52261752"",""universal style Number QF1752""]","xs,s,m,l,xl","xs,s,m,l,xl",black,a ck id unlined triangle bra and string bikini set featuring a convertible design cotton stretch fabric and a thin logo band revisiting cotton classic styling and exploring a new shape


In [73]:
set_cols = ['color', 'descr_conc', 'pdp_url']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,508,1222,3742,422,380,201,22,42,101,677,283,599
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,23,0,0,0,1,0,0,0,0,0,0,0


The Topshop offers two items (a set) on the same page what explains why products have the same description and pdp_url but different product names. The same happens with Calvin Klein.

In [74]:
#topshop
df = brands[3]
index = df.duplicated(set_cols)
#ind = df.index[index][0]
#display(df[(df['pdp_url']==df.loc[ind]['pdp_url'])])
#calvin klein
df = brands[0]
index = df.duplicated(set_cols)
ind = df.index[index][0]
display(df[(df['description']==df.loc[ind]['description'])&(df['pdp_url']==df.loc[ind]['pdp_url'])])

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
4603,modern cotton thong,20.0,20.0,http://www.calvinklein.us/en/womens-clothing/womens-sports-bras/womens-underwear-set-modern-cotton-black-bandeau-1,calvin klein,bralettes,calvin klein us,"a modern cotton bandeau and thong set designed with cotton modal stretch fabric, a wide elasticized logo band and seaming details for iconic, signature styling.",NaN,NaN,"[""soft thong panty"",""elasticized logo waistband"",""cotton gusset"",""tagless"",""tonal topstitching"",""seaming details"",""machine wash"",""53% cotton 35% modal 12% elastane"",""imported"",""Style Number 51473786"",""universal style Number F3786""]","s,m,l,xs,xl","xs,s,m,l,xl",black,a modern cotton bandeau and thong set designed with cotton modal stretch fabric a wide elasticized logo band and seaming details for iconic signature styling
4602,modern cotton bandeau,24.0,24.0,http://www.calvinklein.us/en/womens-clothing/womens-sports-bras/womens-underwear-set-modern-cotton-black-bandeau-1,calvin klein,bralettes,calvin klein us,"a modern cotton bandeau and thong set designed with cotton modal stretch fabric, a wide elasticized logo band and seaming details for iconic, signature styling.",NaN,NaN,"[""modern cotton bandeau"",""cotton modal stretch"",""bandeau shaping"",""wide elasticized logo band"",""tagless"",""tonal topstitching"",""seaming details"",""machine wash"",""53% cotton 35% modal 12% elastane"",""imported"",""Style Number 52264619"",""universal style Number QF4619""]","s,m,l,xs,xl","s,m,l",black,a modern cotton bandeau and thong set designed with cotton modal stretch fabric a wide elasticized logo band and seaming details for iconic signature styling


Wacoal introduced a whitespace in the product name and changed the price.

In [75]:
df = brands[4]
index = df.duplicated(set_cols)
ind = df.index[index][0]
display(df[(df['description']==df.loc[ind]['description'])&(df['pdp_url']==df.loc[ind]['pdp_url'])&(df['color']==df.loc[ind]['color'])])

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
113,b.provocative bikini,19.0,11.4,http://btemptd.wacoal-america.com/b-provocative-bikini-btemptd-943222-p.html,wacoal,panties,btemptd us,"a classic silhouette with timeless appliqued lace and modern stretch fabric with an opalescent sheen, this style can’t help but entice.",NaN,NaN,"[Minimum coverage bikini ,Appliqued lace front and sides on nude stretch opalescent fabric,Pretty detailed edge on leg openings ,Sewn-in cotton panel ,Matching bras 951222, 953222, 910222 (sold separately), 77% Nylon/ 23% Spandex; Panel Lining: 100% cotton ,Hand wash warm,Wash colors separately,Use non-chlorine bleach only when needed,Hang to dry,Do not iron,Do not dry clean]","s,m,l,xl","l,m,s,xl",night,a classic silhouette with timeless appliqued lace and modern stretch fabric with an opalescent sheen this style can’t help but entice
1203,b. provocative bikini,19.0,13.3,http://btemptd.wacoal-america.com/b-provocative-bikini-btemptd-943222-p.html,wacoal,panties,btemptd us,"a classic silhouette with timeless appliqued lace and modern stretch fabric with an opalescent sheen, this style can’t help but entice.",NaN,NaN,"[Minimum coverage bikini ,Appliqued lace front and sides on nude stretch opalescent fabric,Pretty detailed edge on leg openings ,Sewn-in cotton panel ,Matching bras 951222, 953222, 910222 (sold separately), 77% Nylon/ 23% Spandex; Panel Lining: 100% cotton ,Hand wash warm,Wash colors separately,Use non-chlorine bleach only when needed,Hang to dry,Do not iron,Do not dry clean]","s,m,l,xl","l,m,s,xl",night,a classic silhouette with timeless appliqued lace and modern stretch fabric with an opalescent sheen this style can’t help but entice


In [76]:
df = brands[4]
df = df.sort_values(by= 'price')
brands[4] = df.drop_duplicates(set_cols)

Victoria's Secret changed a bit the naming of the product and also reduced the price. We will delete the duplicate with the largest price.

In [77]:
#df = brands[2]
#index = df.duplicated(set_cols)
#ind = df.index[index][7]
#display(df[(df['description']==df.loc[ind]['description'])&(df['pdp_url']==df.loc[ind]['pdp_url'])&(df['color']==df.loc[ind]['color'])])

In [78]:
#df = brands[2]
#df = df.sort_values(by= 'price')
#brands[2] = df.drop_duplicates(set_cols)

The Topshop offers two items (a set) on the same page what explains why products have the same description and pdp_url but different porudct names.

In [79]:
#topshop
try: 
    df = brands[3]
    index = df.duplicated(set_cols)
    ind = df.index[index][0]
    display(df[(df['description']==df.loc[ind]['description'])&(df['pdp_url']==df.loc[ind]['pdp_url'])])
except:
    print('Error')

Error


### 1.3.8 Find records with different price (or mrp), product category and pdp_url
In some brands if we change the product_category the url will also be changed, so we will not take url into consideration. Also, meanwhile a price or mrp could have changed.

In [80]:
#price changed
set_cols = ['descr_conc','product_name','color','mrp']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,508,1222,3742,422,379,201,22,42,101,677,283,599
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,0,6,5,0,0,0,0,0,0,1,0,13


Hanky Panky has the same problem that the same thongs are in collections and thongs category.

In [81]:
df = brands[1]
index = df.duplicated(set_cols)
ind = df.index[index][0]
df[(df.loc[ind].product_name == df.product_name)&(df.loc[ind].color == df.color)&(df.loc[ind].descr_conc == df.descr_conc)]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
12576,breton stripe low rise thong,24.0,14.99,http://www.hankypanky.com/collections/breton-stripe-low-rise-thong.html,hankypanky,collections,hankypanky us,blue and white feels oh so right.,NaN,NaN,"[""One-size thong in signature stretch lace "", "" Fits sizes 2-12 best (hips measuring 35”-42”) "", "" Low rise fits low on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" Body: 100% Nylon; Trim 90% Nylon 10% Spandex; Crotch Lining: 100% Supima® Cotton "", "" Made in the USA""]","[""one size""]","[""one size""]",breton stripe,blue and white feels oh so right
20847,breton stripe low rise thong,24.0,16.99,http://www.hankypanky.com/thongs/breton-stripe-low-rise-thong.html,hankypanky,thongs,hankypanky us,blue and white feels oh so right.,NaN,NaN,"[""One-size thong in signature stretch lace "", "" Fits sizes 2-12 best (hips measuring 35”-42”) "", "" Low rise fits low on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" Body: 100% Nylon; Trim 90% Nylon 10% Spandex; Crotch Lining: 100% Supima® Cotton "", "" Made in the USA""]","[""one size""]","[""one size""]",breton stripe,blue and white feels oh so right


In [82]:
df = brands[11]
index = df.duplicated(set_cols)
ind = df.index[index][0]
df[(df.loc[ind].product_name == df.product_name)&(df.loc[ind].color == df.color)&(df.loc[ind].descr_conc == df.descr_conc)]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
6118,b smooth high cut brief 834175,15.0,10.5,http://www1.macys.com/shop/product/wacoal-b-smooth-high-cut-brief-834175?id=499057&categoryid=55805,wacoal,women - lingerie & shapewear - panties,macys us,"seamless, smooth styling makes this high-cut b-smooth wacoal brief invisible under everything you wear. it features a seamless waistband for comfort. style #834175",4.7,125.0,"[""Lined at gusset"", ""Nylon/polyamide/spandex; crotch lining: Nylon/polyamide/cotton/spandex"", ""Hand wash"", ""Imported"", ""Web ID: 499057""]","[""s"", ""m"", ""l"", ""xl"", ""xxl"", ""2x""]","[""s"", ""l"", ""xl"", ""2x""]",lavender frost,seamless smooth styling makes this high-cut b-smooth wacoal brief invisible under everything you wear it features a seamless waistband for comfort style #834175
5640,b smooth high cut brief 834175,15.0,15.0,http://www1.macys.com/shop/product/wacoal-b-smooth-high-cut-brief-834175?id=499057&categoryid=65739,wacoal,women - lingerie & shapewear - designer lingerie,macys us,"seamless, smooth styling makes this high-cut b-smooth wacoal brief invisible under everything you wear. it features a seamless waistband for comfort. style #834175",4.7,125.0,"[""Lined at gusset"", ""Nylon/polyamide/spandex; crotch lining: Nylon/polyamide/cotton/spandex"", ""Hand wash"", ""Imported"", ""Web ID: 499057""]","[""s"", ""m"", ""l"", ""xl"", ""xxl"", ""2x""]","[""xl""]",lavender frost,seamless smooth styling makes this high-cut b-smooth wacoal brief invisible under everything you wear it features a seamless waistband for comfort style #834175


In [83]:
brands_sorted = []
for brand in brands:
    #so that the more discounted item stays
    brands_sorted.append(brand.sort_values(by='price'))
brands = drop_duplicated_records(brands_sorted, set_cols)

In [84]:
#mrp changed
#set_cols = ['descr_conc','product_name','color','price']
#display(count_duplicates(brands, brands_name, set_cols).transpose())

### 1.3.9 Find records with different price and mrp, product category and pdp_url
Those one are the weirdest ones because the mrp changes as well what actually shoud not be the case. (Or maybe brands have different mrp for different sizes what is also reasonable) We wil delete such records completely.

In [85]:
#price changed
set_cols = ['descr_conc','product_name','color']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,508,1216,3737,422,379,201,22,42,101,676,283,586
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,1,0,75,0,0,41,1,23,42,0,0,0


In [86]:
df = brands[0]
index = df.duplicated(set_cols)
ind = df.index[index][0]
df[(df.loc[ind].product_name == df.product_name)&(df.loc[ind].color == df.color)&(df.loc[ind].descr_conc == df.descr_conc)]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
150,perfectly fit modern t shirt bra,48.0,19.99,http://www.calvinklein.us/en/womens-clothing/womens-bras/perfectly-fit-modern-t-shirt-bra-52083837,calvin klein,bras,calvin klein us,the perfectly fit modern t-shirt bra features full coverage and adjustable straps.,NaN,NaN,"[""perfectly fit t-shirt bra"",""elasticized band"",""full coverage"",""adjustable straps"",""adjustable cushion closure"",""hand wash"",""71% nylon 29% spandex"",""imported""]","32a,32b,32c,32d,32dd,34a,34b,34c,34d,34dd,36a,36b,36c,36d,36dd,38b,38c,38d",34c,black,the perfectly fit modern t-shirt bra features full coverage and adjustable straps
938,perfectly fit modern t shirt bra,46.0,46.00,http://www.calvinklein.us/en/womens-clothing/womens-bras/perfectly-fit-modern-t-shirt-bra-52013837-100,calvin klein,bras,calvin klein us,the perfectly fit modern t-shirt bra features full coverage and adjustable straps.,NaN,NaN,"[""perfectly fit t-shirt bra"",""elasticized band"",""full coverage"",""adjustable straps"",""adjustable cushion closure"",""hand wash"",""71% nylon 29% spandex"",""imported""]","30c,30d,30dd,32a,32b,32c,32d,32dd,32ddd,34a,34b,34c,34d,34dd,34ddd,36a,36b,36c,36d,36dd,36ddd,38b,38c,38d,38dd","30d,30dd,32ddd,34a,34b,34c,34d,34dd,34ddd,36a,36b,36c,36d,36ddd,38dd",black,the perfectly fit modern t-shirt bra features full coverage and adjustable straps


In [87]:
df = brands[2]
index = df.duplicated(set_cols)
ind = df.index[index][0]
df[(df.loc[ind].product_name == df.product_name)&(df.loc[ind].color == df.color)&(df.loc[ind].descr_conc == df.descr_conc)]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
31075,very sexy lace trim cheeky panty,12.5,5.99,https://www.victoriassecret.com/panties/shop-all-panties/lace-trim-cheeky-panty-very-sexy?productid=335969&cataloguetype=ols,victoria's secret,lace-trim cheeky panty,victoriassecret us,"a sweet way to show a little cheek. trimmed in supersoft lace, this sexy little panty comes in a rainbow of colors and prints–why not flirt in them all? front bow and lace trim low rise minimal back coverage imported polyester/spandex",4.6,118.0,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",s,cadette green,a sweet way to show a little cheek trimmed in supersoft lace this sexy little panty comes in a rainbow of colors and prints–why not flirt in them all? front bow and lace trim low rise minimal back coverage imported polyester/spandex
347188,very sexy lace trim cheeky panty,14.5,14.50,https://www.victoriassecret.com/panties/shop-all-panties/lace-trim-cheeky-panty-very-sexy?productid=313571&cataloguetype=ols,victoria's secret,lace-trim cheeky panty,victoriassecret us,"a sweet way to show a little cheek. trimmed in supersoft lace, this sexy little panty comes in a rainbow of colors and prints–why not flirt in them all? front bow and lace trim low rise minimal back coverage imported polyester/spandex",4.6,121.0,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",s,cadette green,a sweet way to show a little cheek trimmed in supersoft lace this sexy little panty comes in a rainbow of colors and prints–why not flirt in them all? front bow and lace trim low rise minimal back coverage imported polyester/spandex


In [88]:
df = brands[5]
index = df.duplicated(set_cols)
ind = df.index[index][0]
df[(df.loc[ind].product_name == df.product_name)&(df.loc[ind].color == df.color)&(df.loc[ind].descr_conc == df.descr_conc)]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
18087,calvin klein women's bottoms up bikini panty,7.51,7.51,https://www.amazon.com/-/dp/b01ly0zjbt?th=1&psc=1,calvin-klein,panties,amazon us,"the bottoms up bikini features color prints, a refined lace trim, and a thin waistband for comfort and shape retention.",4.2,121,"[ 82% Nylon, 18% Elastane , Imported , Machine Wash , Calvin Klein logo on waistband , delicate lace trim is a soft design that adds another layer to an everyday basic , fit flexible with stretch , Cotton Gusset; Light Weight; Silky Touch ]","small , medium , large , x-large",medium,desert sunset,the bottoms up bikini features color prints a refined lace trim and a thin waistband for comfort and shape retention
18093,calvin klein women's bottoms up bikini panty,12.00,8.68,https://www.amazon.com/-/dp/b01m0ygbwv?th=1&psc=1,calvin-klein,panties,amazon us,"the bottoms up bikini features color prints, a refined lace trim, and a thin waistband for comfort and shape retention.",4.2,121,"[ 82% Nylon, 18% Elastane , Imported , Machine Wash , Calvin Klein logo on waistband , delicate lace trim is a soft design that adds another layer to an everyday basic , fit flexible with stretch , Cotton Gusset; Light Weight; Silky Touch ]","small , medium , large , x-large",small,desert sunset,the bottoms up bikini features color prints a refined lace trim and a thin waistband for comfort and shape retention


In [89]:
brands[2] = brands[2].drop_duplicates(set_cols, keep = False)
brands[0] = brands[0].drop_duplicates(set_cols, keep = False)
brands[5] = brands[5].drop_duplicates(set_cols, keep = False)
brands[6] = brands[6].drop_duplicates(set_cols, keep = False)
brands[7] = brands[7].drop_duplicates(set_cols, keep = False)
brands[8] = brands[8].drop_duplicates(set_cols, keep = False)

### 1.3.10 When they changed a description a lot

In [90]:
set_cols = ['product_name','color', 'mrp', 'total_sizes', 'price', 'product_category']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,506,1216,3587,422,379,129,20,11,39,676,283,586
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,3,0,26,0,0,0,0,0,0,0,0,0


In [91]:
df = brands[2]
index = df.duplicated(set_cols)
ind = df.index[index][0]
#print(ind)
#display(df.loc[90])
df[(df.loc[ind].product_name == df.product_name)&(df.loc[ind].mrp == df.mrp)&(df.loc[ind].color == df.color)]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
188948,pink lace trim boyshort,10.5,10.5,https://www.victoriassecret.com/pink/panties/lace-trim-boyshort-pink?productid=316664&cataloguetype=ols,victoria's secret pink,lace trim boyshort,victoriassecret us,"comfy, classic, and perfect for lounging this low rise cotton boyshort features a pretty lace waistband. narrow lace trim extra low rise boyshort fit cotton/polyamide/elastane",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l""]",s,watercolor,comfy classic and perfect for lounging this low rise cotton boyshort features a pretty lace waistband narrow lace trim extra low rise boyshort fit cotton/polyamide/elastane
3557,pink lace trim boyshort,10.5,10.5,https://www.victoriassecret.com/pink/panties/lace-trim-boyshort-pink?productid=313317&cataloguetype=ols,victoria's secret pink,lace trim boyshort,victoriassecret us,get in the holiday spirit with this sweet boyshort! features a lace trim waistband. lace-trim waistband extra low rise imported cotton/polyamide/elastane,NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l""]",s,watercolor,get in the holiday spirit with this sweet boyshort! features a lace trim waistband lace-trim waistband extra low rise imported cotton/polyamide/elastane


In [92]:
import sys
try:
    df = brands[3]
    index = df.duplicated(set_cols)
    ind = df.index[index][0]
    df[(df.loc[ind].product_name == df.product_name)&(df.loc[ind].mrp == df.mrp)]
except:
    print("Unexpected error:", sys.exc_info()[0])

Unexpected error: <class 'IndexError'>


In [93]:
brands = drop_duplicated_records(brands, set_cols)

Price changed and a bit of description

In [94]:
set_cols = ['product_name','color', 'total_sizes', 'product_category', 'mrp']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,503,1216,3561,422,379,129,20,11,39,676,283,586
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,3,0,30,1,0,3,0,0,0,0,0,0


In [95]:
df = brands[2]
index = df.duplicated(set_cols)
ind = df.index[index][0]
df[(df.loc[ind].product_name == df.product_name)&(df.loc[ind].mrp == df.mrp)&(df.loc[ind].color == df.color)]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
96811,cotton lingerie ruched back hiphugger panty,8.5,3.99,https://www.victoriassecret.com/panties/shop-all-panties/ruched-back-hiphugger-panty-cotton-lingerie?productid=336099&cataloguetype=ols,victoria's secret,ruched-back hiphugger panty,victoriassecret us,"this comfy hiphugger is naturally sexy in silky-soft, stretchy cotton, with flirty back ruching.in lots of colors and pretty prints, our cotton undies ensure your lingerie drawer is anything but ordinary. mid rise moderate back coverage imported cotton/lycra® spandex; pure cotton panel",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",s,retro diamond geo print,this comfy hiphugger is naturally sexy in silky-soft stretchy cotton with flirty back ruching in lots of colors and pretty prints our cotton undies ensure your lingerie drawer is anything but ordinary mid rise moderate back coverage imported cotton/lycra® spandex; pure cotton panel
337785,cotton lingerie ruched back hiphugger panty,8.5,8.50,https://www.victoriassecret.com/panties/shop-all-panties/ruched-back-hiphugger-panty-cotton-lingerie?productid=328519&cataloguetype=ols,victoria's secret,ruched-back hiphugger panty,victoriassecret us,"this comfy hiphugger is naturally sexy in silky-soft, stretchy cotton, with flirty back ruching. in lots of colors and pretty prints, our cotton undies ensure your lingerie drawer is anything but ordinary. picot trim mid rise moderate back coverage imported cotton/elastane/nylon",4.5,124.0,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",s,retro diamond geo print,this comfy hiphugger is naturally sexy in silky-soft stretchy cotton with flirty back ruching in lots of colors and pretty prints our cotton undies ensure your lingerie drawer is anything but ordinary picot trim mid rise moderate back coverage imported cotton/elastane/nylon


In [96]:
brands_sorted = []
for brand in brands:
    #so that the more discounted item stays
    brands_sorted.append(brand.sort_values(by='price'))
brands = drop_duplicated_records(brands_sorted, set_cols)

MRP changed

In [97]:
set_cols = ['product_name','color', 'total_sizes', 'product_category', 'price']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,500,1216,3531,421,379,126,20,11,39,676,283,586
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,0,0,1,0,0,0,0,0,0,0,0,0


In [98]:
df = brands[2]
index = df.duplicated(set_cols)
ind = df.index[index][0]
df[(df.loc[ind].product_name == df.product_name)&(df.loc[ind].price == df.price)&(df.loc[ind].color == df.color)]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
39110,body by victoria demi bra,44.5,19.99,https://www.victoriassecret.com/bras/shop-all-bras/demi-bra-body-by-victoria?productid=336236&cataloguetype=ols,victoria's secret,demi bra,victoriassecret us,"a lower cut makes this supersoft bra perfect for revealing necklines, with lighter memory fit for extra support as it conforms to your curves and a smoothing u-shaped back. lift & lining lightly lined for subtle lift underwire cups straps & hooks adjustable straps can convert to crossback and snap into place for a secure hold back closure double row of hook and eye closures; sizes 34ddd, 36dd-36ddd & 38d-38ddd have triple row of closures; sizes 40dd and 40ddd have 4 rows of closure for a secure, comfortable fit 4 settings to ensure a perfect fit details & fabric u-shaped ballet back prevents band from riding up and offers more coverage double-lined sides for the smoothest shape keyhole and bow on center front imported nylon/spandex",NaN,NaN,NaN,"[""30a"", ""30b"", ""30c"", ""30d"", ""30dd"", ""30ddd"", ""32a"", ""32b"", ""32c"", ""32d"", ""32dd"", ""32ddd"", ""34a"", ""34b"", ""34c"", ""34d"", ""34dd"", ""34ddd"", ""36a"", ""36b"", ""36c"", ""36d"", ""36dd"", ""36ddd"", ""38a"", ""38b"", ""38c"", ""38d"", ""38dd"", ""38ddd"", ""40a"", ""40b"", ""40c"", ""40d"", ""40dd"", ""40ddd""]",34a,faded denim,a lower cut makes this supersoft bra perfect for revealing necklines with lighter memory fit for extra support as it conforms to your curves and a smoothing u-shaped back lift & lining lightly lined for subtle lift underwire cups straps & hooks adjustable straps can convert to crossback and snap into place for a secure hold back closure double row of hook and eye closures; sizes 34ddd 36dd-36ddd & 38d-38ddd have triple row of closures; sizes 40dd and 40ddd have 4 rows of closure for a secure comfortable fit 4 settings to ensure a perfect fit details & fabric u-shaped ballet back prevents band from riding up and offers more coverage double-lined sides for the smoothest shape keyhole and bow on center front imported nylon/spandex
110,body by victoria demi bra,54.5,19.99,https://www.victoriassecret.com/bras/shop-all-bras/demi-bra-body-by-victoria?productid=338118&cataloguetype=ols,victoria's secret,demi bra,victoriassecret us,"sexy comfort and a sleek shape start with low-cut cups perfect for revealing necklines. lighter memory fit lining for extra support as it conforms to your curves features a smoothing u-shaped back. lift & lining lightly lined for shape underwire cups straps & hooks adjustable straps can convert to crossback and snap into place for a secure hold back closure double row of hook and eye closures; sizes 34ddd, 36dd-36ddd & 38d-38ddd have triple row of closures for a secure, comfortable fit 4 settings to ensure a perfect fit details & fabric u-shaped ballet back prevents band from riding up and offers more coverage double-lined sides for a smooth shape embossed vs logo at back hook & eye closure imported nylon/spandex",NaN,NaN,NaN,"[""30a"", ""30b"", ""30c"", ""30d"", ""30dd"", ""30ddd"", ""32a"", ""32b"", ""32c"", ""32d"", ""32dd"", ""32ddd"", ""34a"", ""34b"", ""34c"", ""34d"", ""34dd"", ""34ddd"", ""36a"", ""36b"", ""36c"", ""36d"", ""36dd"", ""36ddd"", ""38a"", ""38b"", ""38c"", ""38d"", ""38dd"", ""38ddd"", ""40a"", ""40b"", ""40c"", ""40d"", ""40dd"", ""40ddd""]",34a,faded denim,sexy comfort and a sleek shape start with low-cut cups perfect for revealing necklines lighter memory fit lining for extra support as it conforms to your curves features a smoothing u-shaped back lift & lining lightly lined for shape underwire cups straps & hooks adjustable straps can convert to crossback and snap into place for a secure hold back closure double row of hook and eye closures; sizes 34ddd 36dd-36ddd & 38d-38ddd have triple row of closures for a secure comfortable fi

In [99]:
brands[2] = brands[2].drop_duplicates(set_cols, keep = False)

In [100]:
#description changed and product category
set_cols = ['product_name','pdp_url', 'mrp', 'price', 'color']
display(count_duplicates(brands, brands_name, set_cols).transpose())

,0,0,0,0,0,0,0,0,0,0,0,0
Number of recs,500,1216,3529,421,379,126,20,11,39,676,283,586
brand,Calvin Klein,Hanky Panky,Victoria's Secret,Topshop,Wacoal,Amazon Calvin Klein,Amazon Hanky Panky,Amazon Victoria's Secret,Amazon Wacoal,Macys Calvin Klein,Macys Hanky Panky,Macys Wacoal
duplicates,0,0,0,0,2,0,0,0,0,0,0,0


In [101]:
df = brands[4]
index = df.duplicated(set_cols)
ind = df.index[index][0]
df[(df.loc[ind].product_name == df.product_name)&(df.loc[ind].price == df.price)&(df.loc[ind].color == df.color)]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
155,ciao bella wire free bralette,32.0,32.0,http://btemptd.wacoal-america.com/ciao-bella-wire-free-bralette-btemptd-916144-p.html,wacoal,collections,btemptd us,a dote-worthy delicate bralette that softly shapes and supports. corded lace cups paired with a dramatic scallop detail at the neckline make this style unabashedly feminine (and quite flirtatious!),NaN,NaN,"[• Moderate coverage wire free bralette• Triangle cups with vertical seam for added shaping • Corded lace cups lined with stretch mesh• Dramatic scallop detail at neckline • Lace covered band anchors and supports• Hook and eye back closure • Fully back adjustable stretch straps • Matching panties 976144, 978144 and 945144 (sold separately), Body: 86% Nylon/14% Spandex ,Hand wash warm,Wash colors separately,Use non-chlorine bleach only when needed,Hang to dry,Do not iron,Do not dry clean]","s,m,l,xl","l,m,s,xl",vanilla ice,a dote-worthy delicate bralette that softly shapes and supports corded lace cups paired with a dramatic scallop detail at the neckline make this style unabashedly feminine (and quite flirtatious!)
1587,ciao bella wire free bralette,32.0,32.0,http://btemptd.wacoal-america.com/ciao-bella-wire-free-bralette-btemptd-916144-p.html,wacoal,bras and bralettes,btemptd us,a dote-worthy delicate bralette that softly shapes and supports. corded lace cups paired with a dramatic scallop detail at the necklace make this style unabashedly feminine (and quite flirtatious!),NaN,NaN,"[• Moderate coverage wire free bralette• Triangle cups with vertical seam for added shaping • Corded lace cups lined with stretch mesh• Dramatic scallop detail at neckline • Lace covered band anchors and supports• Hook and eye back closure • Fully back adjustable stretch straps • Matching panties 976144, 978144 and 945144 (sold separately), Body: 86% Nylon/14% Spandex ,Hand wash warm,Wash colors separately,Use non-chlorine bleach only when needed,Hang to dry,Do not iron,Do not dry clean]","s,m,l,xl","l,m,s,xl",vanilla ice,a dote-worthy delicate bralette that softly shapes and supports corded lace cups paired with a dramatic scallop detail at the necklace make this style unabashedly feminine (and quite flirtatious!)


In [102]:
df=brands[4]
df[df['product_name']=='ciao bella wire free bralette'][['mrp','price','color']]

,mrp,price,color
1586,32.0,22.4,camellia rose
155,32.0,32.0,vanilla ice
157,32.0,32.0,wild aster
156,32.0,32.0,night
1587,32.0,32.0,vanilla ice
1588,32.0,32.0,night


In [103]:
brands = drop_duplicated_records(brands, set_cols)

### 1.3.11 When they changed a description, pdp_url and price.
Mainly it is a problem in calvin klein and victoria's secret.

The description was changed because they are selling a set of bra and panties.

In [104]:
df = brands[0]
df[(df['product_name']=='sheer marquisette lace triangle bra')&(df['color']=='perpetua')]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
223,sheer marquisette lace triangle bra,36.0,19.99,http://www.calvinklein.us/en/womens-clothing/womens-bras/sheer-marquisette-lace-triangle-bra-52261842-030,calvin klein,bras,calvin klein us,"a triangle bra featuring supersoft lightweight sheer and modern swing lace trim reintroducing the quintessential ck fabrication, sheer marquisette.",NaN,NaN,"[""sheer marquisette lace triangle bra"",""supersoft lightweight sheer"",""modern swing lace trim"",""revisiting the original sheer marquisette logo"",""triangle design"",""adjustable cushioned straps"",""machine wash"",""72% nylon 28% elastane"",""imported""]","xs,s,m,l,xl","xs,s,m,l",perpetua,a triangle bra featuring supersoft lightweight sheer and modern swing lace trim reintroducing the quintessential ck fabrication sheer marquisette
4621,sheer marquisette lace triangle bra,36.0,36.00,http://www.calvinklein.us/en/womens-clothing/women-sets/sheer-marquisette-lace-triangle-bra--hipster-womens-underwear-set-sheer-marq-triangle-perpetua,calvin klein,panty sets,calvin klein us,"a triangle bra and hipster set designed with supersoft lightweight sheer, modern swing lace trim and subtle logo detailing reintroducing the quintessential ck fabrication, sheer marquisette.",NaN,NaN,"[""sheer marquisette lace triangle bra"",""supersoft lightweight sheer"",""modern swing lace trim"",""revisiting the original sheer marquisette logo"",""triangle design"",""adjustable cushioned straps"",""machine wash"",""72% nylon 28% elastane"",""imported"",""Style Number 52261842-489"",""universal style Number QF1842""]","xs,s,m,l,xl","xs,s,m,l,xl",perpetua,a triangle bra and hipster set designed with supersoft lightweight sheer modern swing lace trim and subtle logo detailing reintroducing the quintessential ck fabrication sheer marquisette


We will find the score of the similarity of two descriptions and decide upon it how similar the products are.

In [105]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [106]:
df_main = pd.DataFrame()
for ind, brand in enumerate(brands):
    df = brand
    df2 = brand
    df = pd.DataFrame(df.groupby(['product_name','color']).size())
    df = df[df[0]>1].reset_index()
    
    df['score'] = None
    df['brand'] = brands_name[ind]
    for ind, row in df.iterrows():
        d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
        score = similar(d.iloc[0].descr_conc, d.iloc[1].descr_conc)
        df.set_value(ind,'score',score)
    df_main = df_main.append(df)
    

In [107]:
df_main.sort_values(by='score')

,product_name,color,0,score,brand
32,modern cotton short,black,2,0.0174419,Calvin Klein
4,very sexy lace cheeky panty,black,2,0.0246407,Victoria's Secret
6,very sexy lace cheeky panty,fir green,2,0.0246407,Victoria's Secret
5,very sexy lace cheeky panty,coconut white,2,0.0246407,Victoria's Secret
7,very sexy lace cheeky panty,teak,2,0.0310078,Victoria's Secret
2,victoria's secret the ultimate maximum support sport bra,pink white colorblock,2,0.0828625,Amazon Victoria's Secret
1,victoria's secret the ultimate maximum support sport bra,magenta teal,2,0.0828625,Amazon Victoria's Secret
0,victoria's secret the ultimate maximum support sport bra,colorblock multi,2,0.0903955,Amazon Victoria's Secret
25,modern cotton bralette,censored logo,2,0.16763,Calvin Klein
17,modern cotton bikini,censored logo,2,0.185629,Calvin Klein


Amazon Victoria's Secret

In [108]:
brand_index = 7
df2 = brands[brand_index]
df_slave = df_main[df_main['brand']==brands_name[brand_index]]
for ind, row in df_slave.iterrows():
    print(row['score'])
    d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
    display(d)

0.0903954802259887


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
1966,victoria's secret the ultimate maximum support sport bra,49.49,49.49,https://www.amazon.com/-/dp/b072fbmxfd?th=1&psc=1,victorias-secret,bras,amazon us,"meet our most intuitive sports bra, with maximum support and sleek comfort for running, boxing, and high-impact work outs. wider double-lined spandex sides provide an ultra smooth shape while an encased plastic underwire flexes with you. adjustable straps can convert to crossback with front clip closures that snap into place.",4.6,58,"[ Adjust-and-go straps! Wear them classic or crossback. , Smoothing technology and perfected fit. , Extra-cushioned closure means no pain during floor exercises. , Body-Wick technology to keep you cool and dry. ]","32d , 32ddd , 34b , 34c , 34d , 34dd , 34ddd , 36c , 36d , 36dd , 36ddd , 38c , 38d , 38dd , 38ddd","34c , 34ddd",colorblock multi,meet our most intuitive sports bra with maximum support and sleek comfort for running boxing and high-impact work outs wider double-lined spandex sides provide an ultra smooth shape while an encased plastic underwire flexes with you adjustable straps can convert to crossback with front clip closures that snap into place
27371,victoria's secret the ultimate maximum support sport bra,49.49,49.49,https://www.amazon.com/-/dp/b07284hlhn?th=1&psc=1,victorias-secret,bras,amazon us,"maximum support, body-wick keeps you cool & dry, concealed, flexible underwire for cushioned support, wear it classic & crossback; with adjustable straps that attach in front, extra-cushioned adjustable back closure",4.4,65,"[ Adjust-and-go straps! Wear them classic or crossback. , Smoothing technology and perfected fit. , Extra-cushioned closure means no pain during floor exercises. , Body-Wick technology to keep you cool and dry. ]","34b , 34c , 34d , 34dd , 34ddd , 36c , 36d , 36dd , 36ddd , 38c , 38d , 38dd , 38ddd",34c,colorblock multi,maximum support body-wick keeps you cool & dry concealed flexible underwire for cushioned support wear it classic & crossback; with adjustable straps that attach in front extra-cushioned adjustable back closure


0.08286252354048965


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
27799,victoria's secret the ultimate maximum support sport bra,36.95,36.95,https://www.amazon.com/-/dp/b01h0ydwg2?th=1&psc=1,victorias-secret,bras,amazon us,"maximum support, body-wick keeps you cool & dry, concealed, flexible underwire for cushioned support, wear it classic & crossback; with adjustable straps that attach in front, extra-cushioned adjustable back closure",4.4,65,"[ Maximum support, Body-Wick keeps you cool & dry , Concealed, flexible underwire for cushioned support , Wear it classic & crossback; with adjustable straps that attach in front , Extra-cushioned adjustable back closure , Nylon performance fabric with four-way stretch ]","34b , 34c , 34d , 34dd , 34ddd , 36c , 36d , 36dd , 36ddd , 38c , 38d , 38dd , 38ddd",36dd,magenta teal,maximum support body-wick keeps you cool & dry concealed flexible underwire for cushioned support wear it classic & crossback; with adjustable straps that attach in front extra-cushioned adjustable back closure
1385,victoria's secret the ultimate maximum support sport bra,39.99,39.99,https://www.amazon.com/-/dp/b01h0ye0k4?th=1&psc=1,victorias-secret,bras,amazon us,"meet our most intuitive sports bra, with maximum support and sleek comfort for running, boxing, and high-impact work outs. wider double-lined spandex sides provide an ultra smooth shape while an encased plastic underwire flexes with you. adjustable straps can convert to crossback with front clip closures that snap into place.",4.6,58,"[ Maximum support, Body-Wick keeps you cool & dry , Concealed, flexible underwire for cushioned support , Wear it classic & crossback; with adjustable straps that attach in front , Extra-cushioned adjustable back closure , Nylon performance fabric with four-way stretch ]","32d , 32ddd , 34b , 34c , 34d , 34dd , 34ddd , 36c , 36d , 36dd , 36ddd , 38c , 38d , 38dd , 38ddd","34b , 36ddd",magenta teal,meet our most intuitive sports bra with maximum support and sleek comfort for running boxing and high-impact work outs wider double-lined spandex sides provide an ultra smooth shape while an encased plastic underwire flexes with you adjustable straps can convert to crossback with front clip closures that snap into place


0.08286252354048965


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
130,victoria's secret the ultimate maximum support sport bra,39.99,39.99,https://www.amazon.com/-/dp/b01lynh2ag?th=1&psc=1,victorias-secret,bras,amazon us,"maximum support, body-wick keeps you cool & dry, concealed, flexible underwire for cushioned support, wear it classic & crossback; with adjustable straps that attach in front, extra-cushioned adjustable back closure",4.4,70,"[ Maximum support, Body-Wick keeps you cool & dry , Concealed, flexible underwire for cushioned support , Wear it classic & crossback; with adjustable straps that attach in front , Extra-cushioned adjustable back closure , Nylon performance fabric with four-way stretch ]","32d , 34b , 34c , 34d , 34dd , 34ddd , 36c , 36d , 36dd , 36ddd , 38c , 38d , 38dd , 38ddd",34d,pink white colorblock,maximum support body-wick keeps you cool & dry concealed flexible underwire for cushioned support wear it classic & crossback; with adjustable straps that attach in front extra-cushioned adjustable back closure
1435,victoria's secret the ultimate maximum support sport bra,55.00,55.00,https://www.amazon.com/-/dp/b01ly01jd6?th=1&psc=1,victorias-secret,bras,amazon us,"meet our most intuitive sports bra, with maximum support and sleek comfort for running, boxing, and high-impact work outs. wider double-lined spandex sides provide an ultra smooth shape while an encased plastic underwire flexes with you. adjustable straps can convert to crossback with front clip closures that snap into place.",4.6,58,"[ Maximum support, Body-Wick keeps you cool & dry , Concealed, flexible underwire for cushioned support , Wear it classic & crossback; with adjustable straps that attach in front , Extra-cushioned adjustable back closure , Nylon performance fabric with four-way stretch ]","32d , 32ddd , 34b , 34c , 34d , 34dd , 34ddd , 36c , 36d , 36dd , 36ddd , 38c , 38d , 38dd , 38ddd",34d,pink white colorblock,meet our most intuitive sports bra with maximum support and sleek comfort for running boxing and high-impact work outs wider double-lined spandex sides provide an ultra smooth shape while an encased plastic underwire flexes with you adjustable straps can convert to crossback with front clip closures that snap into place


Price can vary depending on the size. We will take the most expensive size. If the sizes are the same, we will take the cheapest because sometimes you can find the same product twice on Amazon with different price.

We will delete the ones tWe delete record at index 10 because the same bra as [here](https://www.amazon.com/-/dp/b01ly01jd6?th=1&psc=1) is cheaper [here](https://www.amazon.com/-/dp/b01lynh2ag?th=1&psc=1),

In [109]:
df = brands[brand_index]
df = df[df['pdp_url'].isin(['https://www.amazon.com/-/dp/b01h0ydwg2?th=1&psc=1','https://www.amazon.com/-/dp/b01ly01jd6?th=1&psc=1'])]
index = df.index.tolist()
brands[brand_index] = brands[brand_index].drop(index)

Topshop

In [110]:
brand_index = 3
df2 = brands[brand_index]
df_slave = df_main[df_main['brand']==brands_name[brand_index]]
for ind, row in df_slave.iterrows():
    print(row['score'])
    d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
    display(d)

0.5782312925170068


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
13,embroidered mini knickers,20.0,20.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/embroidered-mini-knickers-6765296,us topshop,lingerie,topshop us,"in a black mesh design, these flattering mini knickers come in a pretty embroidered floral design. 86% polyamide, 14% elastane. machine wash.",NaN,NaN,NaN,"2,4,6,8,10,12,14","2,4,6,8,10,12",black,in a black mesh design these flattering mini knickers come in a pretty embroidered floral design 86% polyamide 14% elastane machine wash
2616,embroidered mini knickers,20.0,20.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/embroidered-mini-knickers-6125457,us topshop,lingerie,topshop us,"opt for a pretty touch when choosing your lingerie with these black mesh mini knickers complete with floral embroidery. 84% polyamide,16% elastane. machine wash.",4.0,1.0,NaN,"2,4,6,8,10,12","2,6,10",black,opt for a pretty touch when choosing your lingerie with these black mesh mini knickers complete with floral embroidery 84% polyamide 16% elastane machine wash


0.5560165975103735


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
218,lace thong,10.0,10.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/lace-thong-6688831,us topshop,lingerie,topshop us,"dark nude knickers finished with pretty lace and mesh. 89% nylon, 11% elastane. machine wash.",NaN,NaN,NaN,"2,4,6,8,10,12,14","2,4,6,8,10,12",dark nude,dark nude knickers finished with pretty lace and mesh 89% nylon 11% elastane machine wash
1189,lace thong,10.0,10.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/lace-thong-6068666,us topshop,lingerie,topshop us,"keep fitted silhouettes seam-free with this pretty nude lace thong underneath. a semi-sheer style with a low rise fit. 89% nylon,11% elastane. machine wash.",NaN,NaN,NaN,"2,4,6,8,10,12",10,dark nude,keep fitted silhouettes seam-free with this pretty nude lace thong underneath a semi-sheer style with a low rise fit 89% nylon 11% elastane machine wash


0.4492753623188406


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
2211,mini knickers by topshop bride,20.0,20.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/mini-knickers-by-topshop-bride-6492198,us topshop,lingerie,topshop us,"elevate your bridal intimates with these pretty ivory knickers in a satin-feel. 96% polyester, 4% elastane. machine wash.",NaN,NaN,NaN,"2,4,6,8,10,12","2,6,8,10,12",ivory,elevate your bridal intimates with these pretty ivory knickers in a satin-feel 96% polyester 4% elastane machine wash
2240,mini knickers by topshop bride,22.0,22.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/mini-knickers-by-topshop-bride-6491987,us topshop,lingerie,topshop us,"in a beautiful ivory lace, these low-rise mini knickers come with good coverage and are perfect for wearing under your wedding dress. 100% polyester. machine wash.",NaN,NaN,NaN,"2,4,6,8,10,12","2,4,6,8,10,12",ivory,in a beautiful ivory lace these low-rise mini knickers come with good coverage and are perfect for wearing under your wedding dress 100% polyester machine wash


Hanky Panky

In [111]:
#hanky panky has this problem that collections and panties are actually the same thing. and description was totally changed
brand_index = 1
df2 = brands[brand_index]
df_slave = df_main[df_main['brand']==brands_name[brand_index]]
for ind, row in df_slave.iterrows():
    print(row['score'])
    d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
    display(d)

0.41025641025641024


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
760,modal sleep set,89.0,89.0,http://www.hankypanky.com/collections/new-summer-modal-sleep-set.html,hankypanky,collections,hankypanky us,sleepy time happiness.,NaN,NaN,"[""Sleep set in our ultra-lightweight and soft modal jersey "", "" V-neck and racer-back with double straps "", "" Shorts sit below the natural waist "", "" Body: 100% MicroModal®, exclusive of trim. "", "" Made in the USA Size Chart Style Guide Fabric Guide""]","[""select"", ""s"", ""m"", ""l""]","[""select"", ""s"", ""m"", ""l""]",black,sleepy time happiness
17539,modal sleep set,94.0,94.0,http://www.hankypanky.com/collections/new-spring-modal-sleep-set.html,hankypanky,collections,hankypanky us,is it bedtime yet?,NaN,NaN,"[""Cami & short set in our featherweight modal with lace detailing"", "" Deep-V of unlined lace on front"", "" Shorts sit below the natural waist"", "" Body: 100% Modal; Exclusive of trim"", "" Made in the USA Size Chart Style Guide Fabric Guide""]","[""s"", ""m"", ""l""]","[""s"", ""m"", ""l""]",black,is it bedtime yet?


0.29310344827586204


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
1899,signature lace low rise thong,20.0,20.0,http://www.hankypanky.com/collections/signature-lace-low-rise-thong.html,hankypanky,collections,hankypanky us,"these are our renowned thongs, but packaged into an adorable little roll.",NaN,NaN,"[""One-size thong in our signature stretch lace "", "" Fits Sizes 2-12 best (hips measuring 35\""-42\"") "", "" Low Rise fits lower on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" No visible panty line (VPL)"", "" Body: 100% Nylon; Trim: 90% Nylon, 10% Spandex; Crotch Lining: 100% Supima® Cotton"", "" Made in the USA""]","[""select"", ""one size""]","[""select"", ""one size""]",ballet pink,these are our renowned thongs but packaged into an adorable little roll
534,signature lace low rise thong,20.0,20.0,http://www.hankypanky.com/thongs/signature-lace-low-rise-thong.html,hankypanky,thongs,hankypanky us,a must-have for every woman's lingerie drawer.,NaN,NaN,"[""One-size thong in our signature stretch lace "", "" Fits sizes 2-12 best (hips measuring 35\""-42\"") "", "" Low Rise fits lower on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" No visible panty line (VPL) "", "" Inducted into The Undie Awards® Hall of Fame"", "" Body: 100% Nylon; Trim: 90% Nylon, 10% Spandex; Crotch Lining: 100% Supima® Cotton "", "" Made in the USA""]","[""select"", ""one size""]","[""select"", ""one size""]",ballet pink,a must-have for every woman's lingerie drawer


0.20689655172413793


...

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
569,signature lace low rise thong,20.0,20.0,http://www.hankypanky.com/thongs/signature-lace-low-rise-thong.html,hankypanky,thongs,hankypanky us,a must-have for every woman's lingerie drawer.,NaN,NaN,"[""One-size thong in our signature stretch lace "", "" Fits sizes 2-12 best (hips measuring 35\""-42\"") "", "" Low Rise fits lower on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" No visible panty line (VPL) "", "" Inducted into The Undie Awards® Hall of Fame"", "" Body: 100% Nylon; Trim: 90% Nylon, 10% Spandex; Crotch Lining: 100% Supima® Cotton "", "" Made in the USA""]","[""select"", ""one size""]","[""select"", ""one size""]",vanilla,a must-have for every woman's lingerie drawer
1934,signature lace low rise thong,20.0,20.0,http://www.hankypanky.com/collections/signature-lace-low-rise-thong.html,hankypanky,collections,hankypanky us,"these are our renowned thongs, but packaged into an adorable little roll.",NaN,NaN,"[""One-size thong in our signature stretch lace "", "" Fits Sizes 2-12 best (hips measuring 35\""-42\"") "", "" Low Rise fits lower on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" No visible panty line (VPL)"", "" Body: 100% Nylon; Trim: 90% Nylon, 10% Spandex; Crotch Lining: 100% Supima® Cotton"", "" Made in the USA""]","[""select"", ""one size""]","[""select"", ""one size""]",vanilla,these are our renowned thongs but packaged into an adorable little roll


0.20689655172413793


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
570,signature lace low rise thong,20.0,20.0,http://www.hankypanky.com/thongs/signature-lace-low-rise-thong.html,hankypanky,thongs,hankypanky us,a must-have for every woman's lingerie drawer.,NaN,NaN,"[""One-size thong in our signature stretch lace "", "" Fits sizes 2-12 best (hips measuring 35\""-42\"") "", "" Low Rise fits lower on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" No visible panty line (VPL) "", "" Inducted into The Undie Awards® Hall of Fame"", "" Body: 100% Nylon; Trim: 90% Nylon, 10% Spandex; Crotch Lining: 100% Supima® Cotton "", "" Made in the USA""]","[""select"", ""one size""]","[""select"", ""one size""]",vivid coral,a must-have for every woman's lingerie drawer
1935,signature lace low rise thong,20.0,20.0,http://www.hankypanky.com/collections/signature-lace-low-rise-thong.html,hankypanky,collections,hankypanky us,"these are our renowned thongs, but packaged into an adorable little roll.",NaN,NaN,"[""One-size thong in our signature stretch lace "", "" Fits Sizes 2-12 best (hips measuring 35\""-42\"") "", "" Low Rise fits lower on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" No visible panty line (VPL)"", "" Body: 100% Nylon; Trim: 90% Nylon, 10% Spandex; Crotch Lining: 100% Supima® Cotton"", "" Made in the USA""]","[""select"", ""one size""]","[""select"", ""one size""]",vivid coral,these are our renowned thongs but packaged into an adorable little roll


0.20689655172413793


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
571,signature lace low rise thong,20.0,20.0,http://www.hankypanky.com/thongs/signature-lace-low-rise-thong.html,hankypanky,thongs,hankypanky us,a must-have for every woman's lingerie drawer.,NaN,NaN,"[""One-size thong in our signature stretch lace "", "" Fits sizes 2-12 best (hips measuring 35\""-42\"") "", "" Low Rise fits lower on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" No visible panty line (VPL) "", "" Inducted into The Undie Awards® Hall of Fame"", "" Body: 100% Nylon; Trim: 90% Nylon, 10% Spandex; Crotch Lining: 100% Supima® Cotton "", "" Made in the USA""]","[""select"", ""one size""]","[""select"", ""one size""]",white,a must-have for every woman's lingerie drawer
1936,signature lace low rise thong,20.0,20.0,http://www.hankypanky.com/collections/signature-lace-low-rise-thong.html,hankypanky,collections,hankypanky us,"these are our renowned thongs, but packaged into an adorable little roll.",NaN,NaN,"[""One-size thong in our signature stretch lace "", "" Fits Sizes 2-12 best (hips measuring 35\""-42\"") "", "" Low Rise fits lower on the hips "", "" The World’s Most Comfortable Thong® "", "" Hanky Panky’s revolutionary and flattering V-front, V-back waistband—our signature design for over 25 years "", "" No visible panty line (VPL)"", "" Body: 100% Nylon; Trim: 90% Nylon, 10% Spandex; Crotch Lining: 100% Supima® Cotton"", "" Made in the USA""]","[""select"", ""one size""]","[""select"", ""one size""]",white,these are our renowned thongs but packaged into an adorable little roll


In [112]:
df = brands[brand_index]
index = df[df['pdp_url'].str.contains('http://www.hankypanky.com/collections/new-summer-modal-sleep-set.html')].index.tolist()
brands[brand_index].loc[index, 'product_name'] = 'modal sleep set summer'

In [113]:
print(len(df))
print(len(df[df['pdp_url'].str.contains('collections')]))

1216
579


In [114]:
#delete all collections as they are repetitive of the original thongs
index = df[df['pdp_url'].str.contains('collections')].index.tolist()
brands[brand_index].drop(index, inplace = True)

Macys Wacoal

In [115]:
brand_index = 11
print(brands_name[brand_index])
df2 = brands[brand_index]
df_slave = df_main[df_main['brand']==brands_name[brand_index]]
for ind, row in df_slave.iterrows():
    print(row['score'])
    d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
    display(d)

Macys Wacoal
0.9878542510121457


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
31781,wireless high impact sports bra 852218,65.0,65.0,http://www1.macys.com/shop/product/wacoal-wireless-high-impact-sports-bra-852218?id=4498364&categoryid=55799,wacoal,women - lingerie & shapewear - bras,macys us,discover total support paired with the comfort of a wireless design when you pull on this high-impact sports bra from wacoal.,NaN,NaN,"[""Adjustable straps"", ""Wireless, gently molded cups minimize bounce"", ""Wide elastic band under bust for support"", ""Mesh detail at straps, center front and back"", ""Moisture-wicking lining"", ""Triple hook-and-eye back closure"", ""Nylon/spandex"", ""Machine washable"", ""Imported"", ""Web ID: 4498364""]","[""32c"", ""32d"", ""32dd"", ""32ddd"", ""34c"", ""34d"", ""34dd"", ""34ddd"", ""36c"", ""36d"", ""36dd"", ""36ddd"", ""36g"", ""38c"", ""38d"", ""38dd"", ""38ddd"", ""38g"", ""40c"", ""40d"", ""40dd"", ""40ddd"", ""40g""]","[""32c"", ""32d"", ""32dd"", ""32ddd"", ""34c"", ""34d"", ""34dd"", ""34ddd""]",black gray,discover total support paired with the comfort of a wireless design when you pull on this high-impact sports bra from wacoal
1423,wireless high impact sports bra 852218,65.0,65.0,http://www1.macys.com/shop/product/wacoal-wireless-high-impact-sports-bra-852218?id=4498364&categoryid=65739,wacoal,women - lingerie & shapewear - designer lingerie,macys us,discover total support paired with the comfort of a wireless design when you put on this high-impact sports bra from wacoal.,NaN,NaN,"[""Adjustable straps"", ""Wireless, gently molded cups minimize bounce"", ""Wide elastic band under bust for support"", ""Mesh detail at straps, center front and back"", ""Moisture-wicking lining"", ""Triple hook-and-eye back closure"", ""Nylon/spandex"", ""Machine washable"", ""Imported"", ""Web ID: 4498364""]","[""32c"", ""32dd"", ""32ddd"", ""34c"", ""34d"", ""34dd"", ""34ddd"", ""36c"", ""36d"", ""36dd"", ""36ddd"", ""36g"", ""38c"", ""38d"", ""38dd"", ""38ddd"", ""38g"", ""40c"", ""40d"", ""40dd"", ""40ddd"", ""40g""]","[""32c"", ""32dd"", ""32ddd"", ""34c"", ""34d"", ""34dd"", ""34ddd"", ""36c""]",black gray,discover total support paired with the comfort of a wireless design when you put on this high-impact sports bra from wacoal


In [116]:
index = df2[(df2['price']==65.0)&(df2['color']=='black/gray')&(df2['pdp_url']=='http://www1.macys.com/shop/product/wacoal-wireless-high-impact-sports-bra-852218?id=4498364&categoryid=55799')].index.tolist()
print(index)
brands[brand_index] = brands[brand_index].drop(index)

[]


Macys Hanky Panky

In [117]:
brand_index = 10
print(brands_name[brand_index])
df2 = brands[brand_index]
df_slave = df_main[df_main['brand']==brands_name[brand_index]]
for ind, row in df_slave.iterrows():
    print(row['score'])
    d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
    display(d)

Macys Hanky Panky
0.4830917874396135


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
562,printed low rise lace thong,24.0,24.0,http://www1.macys.com/shop/product/hanky-panky-printed-low-rise-lace-thong?id=3582555&categoryid=65739,hanky panky,women - lingerie & shapewear - designer lingerie,macys us,beautiful floral lace is fashioned with a playful heart and emoji print in the love note thong from hanky panky.,NaN,NaN,"[""Low rise"", ""V-shaped lace waistband"", ""Soft, signature lace"", ""One size fits most"", ""Lined at gusset"", ""Nylon; gusset: cotton; exclusive of trim"", ""Hand wash"", ""Made in USA"", ""Web ID: 3582555""]","[""os""]","[""os""]",multi,beautiful floral lace is fashioned with a playful heart and emoji print in the love note thong from hanky panky
2360,printed low rise lace thong,25.0,25.0,http://www1.macys.com/shop/product/hanky-panky-printed-low-rise-lace-thong?id=4402215&categoryid=55805,hanky panky,women - lingerie & shapewear - panties,macys us,create a lovely look with the soft lace and floral print of this low-rise thong from hanky panky.,NaN,NaN,"[""Low rise"", ""Elastic lace waistband"", ""Soft lace"", ""One size fits most"", ""Lined at gusset"", ""Nylon; gusset: cotton"", ""Hand wash"", ""Made in USA"", ""Web ID: 4402215""]","[""os""]","[""os""]",multi,create a lovely look with the soft lace and floral print of this low-rise thong from hanky panky


In [118]:
index = df2[(df2['color']=='multi')&(df2['pdp_url'].isin(['http://www1.macys.com/shop/product/hanky-panky-printed-low-rise-lace-thong?id=3582555&categoryid=65739','http://www1.macys.com/shop/product/hanky-panky-printed-low-rise-lace-thong?id=4402215&categoryid=55805']))].index.tolist()
print(index)
brands[brand_index] = brands[brand_index].drop(index)

[562, 2360]


Calvin Klein

Something went wrong with the data here because we have the same product name but different description featuring tringale bra and string. However, the price for string and for a set is the same, which is weird. We will delete records where in description we have 'set'.

In [119]:
def manipul_regex(str_array):
    string = "|".join(str_array)
    return "(^|\s)(" + string + ")(\s|$)"

In [120]:

brand_index = 0
df2 = brands[brand_index]
df_slave = df_main[df_main['brand']==brands_name[brand_index]]
del_index = []
for ind, row in df_slave.iterrows():
    print(row['score'])
    d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
    g = d[d['description'].str.contains(manipul_regex(['bra','bandeau','bralette']) + '(?=.*(bikini underwear set|bikini set|tanga set|trunk set|thong set|hipster set))')]
    del_index.append(g.index.tolist())
    display(d)

0.75


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
404,ck id thin waistband string bikini,20.0,20.0,http://www.calvinklein.us/en/womens-clothing/women-panties/ck-id-thin-waistband-string-bikini-51471754-072,calvin klein,panties,calvin klein us,"a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",NaN,NaN,"[""ck id thin waistband string bikini"",""bikini silhouette"",""thin logo band"",""full gusset"",""rear coverage"",""machine wash"",""95% cotton 5% spandex"",""imported""]","xs,s,m,l,xl","xs,s,m,l,xl",black,a ck id string bikini featuring cotton stretch a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape
4537,ck id thin waistband string bikini,20.0,20.0,http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-ck-id-triangle-black,calvin klein,panty sets,calvin klein us,"a ck id unlined triangle bra and string bikini set featuring a convertible design, cotton stretch fabric and a thin logo band revisiting cotton classic styling and exploring a new shape.",NaN,NaN,"[""ck id thin waistband string bikini"",""bikini silhouette"",""thin logo band"",""full gusset"",""rear coverage"",""machine wash"",""95% cotton 5% spandex"",""imported"",""Style Number 51471754"",""universal style Number QF1754""]","xs,s,m,l,xl","xs,s,m,l,xl",black,a ck id unlined triangle bra and string bikini set featuring a convertible design cotton stretch fabric and a thin logo band revisiting cotton classic styling and exploring a new shape


0.75


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
407,ck id thin waistband string bikini,20.0,9.99,http://www.calvinklein.us/en/womens-clothing/women-panties/ck-id-thin-waistband-string-bikini-51471754-072,calvin klein,panties,calvin klein us,"a ck id string bikini featuring cotton stretch, a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape.",NaN,NaN,"[""ck id thin waistband string bikini"",""bikini silhouette"",""thin logo band"",""full gusset"",""rear coverage"",""machine wash"",""95% cotton 5% spandex"",""imported""]","xs,s,m,l,xl",xs,simple grid print,a ck id string bikini featuring cotton stretch a thin logo band and string detailing revisiting cotton classic styling and exploring a new shape
4620,ck id thin waistband string bikini,20.0,20.00,http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-ck-id-triangle-grid,calvin klein,panty sets,calvin klein us,"a ck id unlined triangle bra and string bikini set featuring a convertible design, cotton stretch fabric and a thin logo band revisiting cotton classic styling and exploring a new shape.",NaN,NaN,"[""ck id thin waistband string bikini"",""bikini silhouette"",""thin logo band"",""full gusset"",""rear coverage"",""machine wash"",""95% cotton 5% spandex"",""imported"",""Style Number 51471754-072"",""universal style Number QF1754""]","xs,s,m,l,xl","xs,m,l,xl",simple grid print,a ck id unlined triangle bra and string bikini set featuring a convertible design cotton stretch fabric and a thin logo band revisiting cotton classic styling and exploring a new shape


0.75


...

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
4637,sheer marquisette lace triangle bra,36.0,36.0,http://www.calvinklein.us/en/womens-clothing/womens-sports-bras/womens-underwear-set-sheer-marq-triangle-black,calvin klein,bralettes,calvin klein us,"a triangle bra and hipster set designed with supersoft lightweight sheer, modern swing lace trim and subtle logo detailing reintroducing the quintessential ck fabrication, sheer marquisette.",NaN,NaN,"[""sheer marquisette lace triangle bra"",""supersoft lightweight sheer"",""modern swing lace trim"",""revisiting the original sheer marquisette logo"",""triangle design"",""adjustable cushioned straps"",""machine wash"",""72% nylon 28% elastane"",""imported"",""Style Number 52261842"",""universal style Number QF1842""]","xs,s,m,l,xl","xs,s",black,a triangle bra and hipster set designed with supersoft lightweight sheer modern swing lace trim and subtle logo detailing reintroducing the quintessential ck fabrication sheer marquisette
220,sheer marquisette lace triangle bra,36.0,36.0,http://www.calvinklein.us/en/womens-clothing/womens-bras/sheer-marquisette-lace-triangle-bra-52261842-030,calvin klein,bras,calvin klein us,"a triangle bra featuring supersoft lightweight sheer and modern swing lace trim reintroducing the quintessential ck fabrication, sheer marquisette.",NaN,NaN,"[""sheer marquisette lace triangle bra"",""supersoft lightweight sheer"",""modern swing lace trim"",""revisiting the original sheer marquisette logo"",""triangle design"",""adjustable cushioned straps"",""machine wash"",""72% nylon 28% elastane"",""imported""]","xs,s,m,l,xl","xs,s",black,a triangle bra featuring supersoft lightweight sheer and modern swing lace trim reintroducing the quintessential ck fabrication sheer marquisette


0.8192771084337349


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
221,sheer marquisette lace triangle bra,36.0,19.99,http://www.calvinklein.us/en/womens-clothing/womens-bras/sheer-marquisette-lace-triangle-bra-52261842-030,calvin klein,bras,calvin klein us,"a triangle bra featuring supersoft lightweight sheer and modern swing lace trim reintroducing the quintessential ck fabrication, sheer marquisette.",NaN,NaN,"[""sheer marquisette lace triangle bra"",""supersoft lightweight sheer"",""modern swing lace trim"",""revisiting the original sheer marquisette logo"",""triangle design"",""adjustable cushioned straps"",""machine wash"",""72% nylon 28% elastane"",""imported""]","xs,s,m,l,xl",xs,dynamic leopard,a triangle bra featuring supersoft lightweight sheer and modern swing lace trim reintroducing the quintessential ck fabrication sheer marquisette
4702,sheer marquisette lace triangle bra,36.0,36.00,http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-sheer-marq-triangle-leopard,calvin klein,panty sets,calvin klein us,"a triangle bra and hipster set designed with supersoft lightweight sheer, modern swing lace trim and subtle logo detailing reintroducing the quintessential ck fabrication, sheer marquisette.",NaN,NaN,"[""sheer marquisette lace triangle bra"",""supersoft lightweight sheer"",""modern swing lace trim"",""revisiting the original sheer marquisette logo"",""triangle design"",""adjustable cushioned straps"",""machine wash"",""72% nylon 28% elastane"",""imported"",""Style Number 52261842-030"",""universal style Number QF1842""]","xs,s,m,l,xl","xs,s",dynamic leopard,a triangle bra and hipster set designed with supersoft lightweight sheer modern swing lace trim and subtle logo detailing reintroducing the quintessential ck fabrication sheer marquisette


0.8192771084337349


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
223,sheer marquisette lace triangle bra,36.0,19.99,http://www.calvinklein.us/en/womens-clothing/womens-bras/sheer-marquisette-lace-triangle-bra-52261842-030,calvin klein,bras,calvin klein us,"a triangle bra featuring supersoft lightweight sheer and modern swing lace trim reintroducing the quintessential ck fabrication, sheer marquisette.",NaN,NaN,"[""sheer marquisette lace triangle bra"",""supersoft lightweight sheer"",""modern swing lace trim"",""revisiting the original sheer marquisette logo"",""triangle design"",""adjustable cushioned straps"",""machine wash"",""72% nylon 28% elastane"",""imported""]","xs,s,m,l,xl","xs,s,m,l",perpetua,a triangle bra featuring supersoft lightweight sheer and modern swing lace trim reintroducing the quintessential ck fabrication sheer marquisette
4621,sheer marquisette lace triangle bra,36.0,36.00,http://www.calvinklein.us/en/womens-clothing/women-sets/sheer-marquisette-lace-triangle-bra--hipster-womens-underwear-set-sheer-marq-triangle-perpetua,calvin klein,panty sets,calvin klein us,"a triangle bra and hipster set designed with supersoft lightweight sheer, modern swing lace trim and subtle logo detailing reintroducing the quintessential ck fabrication, sheer marquisette.",NaN,NaN,"[""sheer marquisette lace triangle bra"",""supersoft lightweight sheer"",""modern swing lace trim"",""revisiting the original sheer marquisette logo"",""triangle design"",""adjustable cushioned straps"",""machine wash"",""72% nylon 28% elastane"",""imported"",""Style Number 52261842-489"",""universal style Number QF1842""]","xs,s,m,l,xl","xs,s,m,l,xl",perpetua,a triangle bra and hipster set designed with supersoft lightweight sheer modern swing lace trim and subtle logo detailing reintroducing the quintessential ck fabrication sheer marquisette


In [121]:
del_index = [item for sublist in del_index for item in sublist]
print(del_index)
brands[brand_index] = brands[brand_index].drop(del_index)

[4537, 4620, 4616, 4697, 4536, 4619, 4615, 4696, 4595, 4659, 4594, 4623, 4658, 4624, 4602, 1231, 1216, 1215, 4603, 1232, 4609, 4601, 4707, 4608, 4600, 4626, 4572, 4591, 4571, 4590, 4638, 4703, 4622, 4637, 4702, 4621]


In [122]:
#the same panties are linked in set and on the official page
df2 = brands[brand_index]
del_index = []
for ind, row in df_slave.iterrows():
    d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
    if d.shape[0]>1:
        print(row['score'])
        g = d[(d['pdp_url'].str.contains('-set-'))|(d['description'].str.contains(manipul_regex(['set'])))]
        del_index.append(g.index.tolist())
        display(d)

0.44571428571428573


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
4597,modern cotton bikini,20.0,20.0,http://www.calvinklein.us/en/womens-clothing/womens-sports-bras/bralette--bikini-womens-underwear-set-modern-cotton-boundless-logo,calvin klein,bralettes,calvin klein us,a bold color with a contrasting elasticized logo waistband for extreme comfort.,NaN,NaN,"[""cotton bikini panty"",""wide elasticized logo waistband"",""full rear coverage"",""full cotton gusset"",""tagless"",""machine wash"",""53% cotton 35% modal 12% elastane"",""imported"",""Style Number 51473787-075"",""universal style Number F3787""]","xs,s,m,l,xl","xs,s,m,l,xl",boundless logo,a bold color with a contrasting elasticized logo waistband for extreme comfort
392,modern cotton bikini,20.0,20.0,http://www.calvinklein.us/en/womens-clothing/women-panties/modern-cotton-bikini-51473787-100,calvin klein,panties,calvin klein us,this modern cotton bikini panty features full rear coverage and a wide elasticized logo waistband.,NaN,NaN,"[""cotton bikini panty"",""wide elasticized logo waistband"",""full rear coverage"",""full cotton gusset"",""tagless"",""machine wash"",""53% cotton 35% modal 12% elastane"",""imported""]","xs,s,m,l,xl","xs,s,m,l,xl",boundless logo,this modern cotton bikini panty features full rear coverage and a wide elasticized logo waistband


0.18562874251497005


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
545,modern cotton bikini,20.0,14.99,http://www.calvinklein.us/en/womens-clothing/women-panties/modern-cotton-bikini-51471881,calvin klein,panties,calvin klein us,"a modern cotton bikini panty featuring mesh stretch fabric, a strike-out logo design, a wide elasticized logo band and full coverage.",NaN,NaN,"[""modern cotton bikini"",""mesh stretch"",""strike-out logo"",""wide elasticized logo band"",""full rear coverage"",""full gusset"",""tagless"",""tonal topstitching"",""seaming details"",""machine wash"",""92% nylon 8% elastane"",""imported""]","xs,s,m,l,xl","xs,m,xl",censored logo,a modern cotton bikini panty featuring mesh stretch fabric a strike-out logo design a wide elasticized logo band and full coverage
4628,modern cotton bikini,20.0,20.00,http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-modern-censored-logo,calvin klein,panty sets,calvin klein us,"crafted in a cotton modal stretch, this edgy underwear set features a strike-out logo design, a racerback, elasticated logo bands and mesh semi-sheer fabric for extreme comfort in a sleek bralette and bikini.",NaN,NaN,"[""modern cotton bikini"",""mesh stretch"",""strike-out logo"",""wide elasticized logo band"",""full rear coverage"",""full gusset"",""tagless"",""tonal topstitching"",""seaming details"",""machine wash"",""92% nylon 8% elastane"",""imported"",""Style Number 51471881"",""universal style Number QF1881""]","xs,s,m,l,xl","xs,s,m,l,xl",censored logo,crafted in a cotton modal stretch this edgy underwear set features a strike-out logo design a racerback elasticated logo bands and mesh semi-sheer fabric for extreme comfort in a sleek bralette and bikini


0.48


...

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
315,modern cotton short,22.0,22.0,http://www.calvinklein.us/en/womens-clothing/women-panties/modern-cotton-short-51443788,calvin klein,panties,calvin klein us,the modern cotton short features a wide elasticized logo waistband and full cotton gusset.,NaN,NaN,"[""cotton short panty"",""wide elasticized logo waistband"",""full cotton gusset"",""tagless"",""machine wash"",""53% cotton 35% modal 12% elastane"",""imported""]","xs,s,m,l,xl","xs,s,m,l,xl",black,the modern cotton short features a wide elasticized logo waistband and full cotton gusset
4718,modern cotton short,40.0,40.0,http://www.calvinklein.us/en/womens-clothing/women-sleepwear/womens-underwear-set-modern-cotton-hoodie,calvin klein,sleepwear,calvin klein us,"an essential modern cotton loungewear set crafted from a soft cotton modal stretch blend in a hoodie and shorts. featuring a drawstring hood, a full zip closure, kangaroo pockets, elasticated signature logo bands and a short inseam for a chic, sleepwear design.",NaN,NaN,"[""cotton short panty"",""wide elasticized logo waistband"",""full cotton gusset"",""tagless"",""machine wash"",""53% cotton 35% modal 12% elastane"",""imported"",""Style Number 54015717"",""universal style Number QS5717""]","s,m,l","s,m",black,an essential modern cotton loungewear set crafted from a soft cotton modal stretch blend in a hoodie and shorts featuring a drawstring hood a full zip closure kangaroo pockets elasticated signature logo bands and a short inseam for a chic sleepwear design


0.5384615384615384


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
316,modern cotton short,22.0,22.0,http://www.calvinklein.us/en/womens-clothing/women-panties/modern-cotton-short-51443788,calvin klein,panties,calvin klein us,the modern cotton short features a wide elasticized logo waistband and full cotton gusset.,NaN,NaN,"[""cotton short panty"",""wide elasticized logo waistband"",""full cotton gusset"",""tagless"",""machine wash"",""53% cotton 35% modal 12% elastane"",""imported""]","xs,s,m,l,xl","xs,s,m,l,xl",grey heather,the modern cotton short features a wide elasticized logo waistband and full cotton gusset
4618,modern cotton short,22.0,22.0,http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-modern-cotton-grey,calvin klein,panty sets,calvin klein us,the modern cotton bra and panty set features contrasting elasticized bands with a repeat logo.,NaN,NaN,"[""cotton short panty"",""wide elasticized logo waistband"",""full cotton gusset"",""tagless"",""machine wash"",""53% cotton 35% modal 12% elastane"",""imported"",""Style Number 51443788"",""universal style Number F3788""]","xs,s,m,l,xl","s,m,l",grey heather,the modern cotton bra and panty set features contrasting elasticized bands with a repeat logo


0.42857142857142855


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
178,modern cotton thong,20.0,20.0,http://www.calvinklein.us/en/womens-clothing/women-panties/modern-cotton-thong-51473786-100,calvin klein,panties,calvin klein us,a modern cotton thong designed with an elasticized logo waistband and a tagless design.,NaN,NaN,"[""soft thong panty"",""elasticized logo waistband"",""cotton gusset"",""tagless"",""tonal topstitching"",""seaming details"",""machine wash"",""53% cotton 35% modal 12% elastane"",""imported""]","xs,s,m,l,xl","s,m,l",bronze,a modern cotton thong designed with an elasticized logo waistband and a tagless design
4728,modern cotton thong,20.0,20.0,http://www.calvinklein.us/en/womens-clothing/womens-sports-bras/womens-underwear-set-modern-cotton-bronze,calvin klein,bralettes,calvin klein us,"crafted with soft cotton modal stretch, this underwear set features a modern cotton bralette and thong with a wide elasticated logo band, racerback straps and minimal coverage for ultimate comfort.",NaN,NaN,"[""soft thong panty"",""elasticized logo waistband"",""cotton gusset"",""tagless"",""tonal topstitching"",""seaming details"",""machine wash"",""53% cotton 35% modal 12% elastane"",""imported"",""Style Number 51473786-251"",""universal style Number F3786""]","xs,s,m,l,xl","s,m,l",bronze,crafted with soft cotton modal stretch this underwear set features a modern cotton bralette and thong with a wide elasticated logo band racerback straps and minimal coverage for ultimate comfort


In [123]:
index = df2[df2['pdp_url']=='http://www.calvinklein.us/en/womens-clothing/women-sleepwear/womens-underwear-set-modern-cotton-hoodie'].index.tolist()
del_index = [item for sublist in del_index for item in sublist]
del_index = list(set(del_index) - set(index))
brands[brand_index].drop(del_index, inplace = True)
brands[brand_index].loc[index,'product_name'] = 'modern cotton sleepwear'
brands[brand_index].loc[index,'product_name']

4718    modern cotton sleepwear
Name: product_name, dtype: object

In [124]:
del_index

[4617,
 4618,
 4627,
 4628,
 4708,
 4581,
 4709,
 4583,
 4645,
 4716,
 4596,
 4597,
 4725,
 4727,
 4728,
 4604]

In [125]:
for ind, brand in enumerate(brands):
    dup_ind = brand.index.duplicated().sum()
    if dup_ind != 0:
        print(brands_name[ind])

Victoria's Secret

In [126]:
brand_index = 2
df2 = brands[brand_index]
del_index = []
df_slave = df_main[df_main['brand']==brands_name[brand_index]]
for ind, row in df_slave.iterrows():
    d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
    d = d.sort_values(by=['mrp','price'], ascending = [False, True])
    if d.shape[0]>1:
        print(row['score'])
        display(d)
        if d.iloc[0].product_name != 'very sexy lace cheeky panty':
            del_index.append(d.iloc[1].name)

0.9649595687331537


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
90457,dream angels push up bra,49.5,29.99,https://www.victoriassecret.com/bras/shop-all-bras/push-up-bra-dream-angels?productid=338126&cataloguetype=ols&colorcode=s40,victoria's secret,push-up bra,victoriassecret us,"our most heavenly push-up ever, with softer memory fit padding for lots of lightweight lift, stretchier lace sides for more comfort and a smoothing u-shaped back. lift & lining •push-up padding •underwire cups straps & hooks •fully adjustable straps convert to crossback and snap into place for a secure hold •double row of hook-and-eye closures; sizes 36-38dd have triple row of hook-and-eye closures for a secure, comfortable fit •4 settings to ensure a perfect fit details & fabric •u-shaped ballet back that prevents band from riding up •imported nylon/spandex",NaN,NaN,NaN,"[""30a"", ""30b"", ""30c""]",30a,cadette green with solid lace,our most heavenly push-up ever with softer memory fit padding for lots of lightweight lift stretchier lace sides for more comfort and a smoothing u-shaped back lift & lining •push-up padding •underwire cups straps & hooks •fully adjustable straps convert to crossback and snap into place for a secure hold •double row of hook-and-eye closures; sizes 36-38dd have triple row of hook-and-eye closures for a secure comfortable fit •4 settings to ensure a perfect fit details & fabric •u-shaped ballet back that prevents band from riding up •imported nylon/spandex
164203,dream angels push up bra,49.5,29.99,https://www.victoriassecret.com/bras/shop-all-bras/push-up-bra-dream-angels?productid=338246&cataloguetype=ols&colorcode=b5d,victoria's secret,push-up bra,victoriassecret us,"our most heavenly push-up ever, with softer memory fit padding for lots of lightweight lift, stretchier lace sides for more comfort and a smoothing u-shaped back. lift & lining push-up padding underwire cups straps & hooks fully adjustable straps convert to crossback and snap into place for a secure hold double row of hook-and-eye closures; sizes 36-38dd have triple row of hook-and-eye closures for a secure, comfortable fit 4 settings to ensure a perfect fit details & fabric u-shaped ballet back that prevents band from riding up imported nylon/spandex",NaN,NaN,NaN,"[""30a"", ""30b"", ""30c"", ""30d"", ""30dd"", ""30ddd"", ""32a"", ""32b"", ""32c"", ""32d"", ""32dd"", ""32ddd"", ""34a"", ""34b"", ""34c"", ""34d"", ""34dd"", ""34ddd"", ""36a"", ""36b"", ""36c"", ""36d"", ""36dd"", ""36ddd"", ""38a"", ""38b"", ""38c"", ""38d"", ""38dd"", ""38ddd""]",36c,cadette green with solid lace,our most heavenly push-up ever with softer memory fit padding for lots of lightweight lift stretchier lace sides for more comfort and a smoothing u-shaped back lift & lining push-up padding underwire cups straps & hooks fully adjustable straps convert to crossback and snap into place for a secure hold double row of hook-and-eye closures; sizes 36-38dd have triple row of hook-and-eye closures for a secure comfortable fit 4 settings to ensure a perfect fit details & fabric u-shaped ballet back that prevents band from riding up imported nylon/spandex


0.9649595687331537


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
90455,dream angels push up bra,49.5,29.99,https://www.victoriassecret.com/bras/shop-all-bras/push-up-bra-dream-angels?productid=338126&cataloguetype=ols&colorcode=s40,victoria's secret,push-up bra,victoriassecret us,"our most heavenly push-up ever, with softer memory fit padding for lots of lightweight lift, stretchier lace sides for more comfort and a smoothing u-shaped back. lift & lining •push-up padding •underwire cups straps & hooks •fully adjustable straps convert to crossback and snap into place for a secure hold •double row of hook-and-eye closures; sizes 36-38dd have triple row of hook-and-eye closures for a secure, comfortable fit •4 settings to ensure a perfect fit details & fabric •u-shaped ballet back that prevents band from riding up •imported nylon/spandex",NaN,NaN,NaN,"[""30a"", ""30b"", ""30c""]",30a,silver sea with solid lace,our most heavenly push-up ever with softer memory fit padding for lots of lightweight lift stretchier lace sides for more comfort and a smoothing u-shaped back lift & lining •push-up padding •underwire cups straps & hooks •fully adjustable straps convert to crossback and snap into place for a secure hold •double row of hook-and-eye closures; sizes 36-38dd have triple row of hook-and-eye closures for a secure comfortable fit •4 settings to ensure a perfect fit details & fabric •u-shaped ballet back that prevents band from riding up •imported nylon/spandex
164201,dream angels push up bra,49.5,29.99,https://www.victoriassecret.com/bras/shop-all-bras/push-up-bra-dream-angels?productid=338246&cataloguetype=ols&colorcode=b5d,victoria's secret,push-up bra,victoriassecret us,"our most heavenly push-up ever, with softer memory fit padding for lots of lightweight lift, stretchier lace sides for more comfort and a smoothing u-shaped back. lift & lining push-up padding underwire cups straps & hooks fully adjustable straps convert to crossback and snap into place for a secure hold double row of hook-and-eye closures; sizes 36-38dd have triple row of hook-and-eye closures for a secure, comfortable fit 4 settings to ensure a perfect fit details & fabric u-shaped ballet back that prevents band from riding up imported nylon/spandex",NaN,NaN,NaN,"[""30a"", ""30b"", ""30c"", ""30d"", ""30dd"", ""30ddd"", ""32a"", ""32b"", ""32c"", ""32d"", ""32dd"", ""32ddd"", ""34a"", ""34b"", ""34c"", ""34d"", ""34dd"", ""34ddd"", ""36a"", ""36b"", ""36c"", ""36d"", ""36dd"", ""36ddd"", ""38a"", ""38b"", ""38c"", ""38d"", ""38dd"", ""38ddd""]",36c,silver sea with solid lace,our most heavenly push-up ever with softer memory fit padding for lots of lightweight lift stretchier lace sides for more comfort and a smoothing u-shaped back lift & lining push-up padding underwire cups straps & hooks fully adjustable straps convert to crossback and snap into place for a secure hold double row of hook-and-eye closures; sizes 36-38dd have triple row of hook-and-eye closures for a secure comfortable fit 4 settings to ensure a perfect fit details & fabric u-shaped ballet back that prevents band from riding up imported nylon/spandex


0.9649595687331537


...

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
266435,victoria sport the player by victoria sport plunge sport bra,24.5,12.99,https://www.victoriassecret.com/bras/shop-all-bras/the-player-by-victoria-sport-plunge-sport-bra-victoria-sport?productid=331806&cataloguetype=ols,victoria's secret,the player by victoria sport plunge sport bra,victoriassecret us,"pro-support, anti-smash: this sport bra teams a plunge front with a racerback and medium support. details medium support. pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning, kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash. tumble dry. imported polyester/spandex",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",xl,black,pro-support anti-smash: this sport bra teams a plunge front with a racerback and medium support details medium support pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash tumble dry imported polyester/spandex
239026,victoria sport the player by victoria sport plunge sport bra,24.5,24.50,https://www.victoriassecret.com/victorias-secret-sport/shop-all/the-player-by-victoria-sport-plunge-sport-bra-victoria-sport?productid=328160&cataloguetype=ols,victoria's secret,the player by victoria sport plunge sport bra,victoriassecret us,"pro-support, anti-smash: this sport bra teams a plunge front with a racerback and light.details medium support. pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning, kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash. tumble dry. imported polyester/spandex care: machine wash. tumble dry. origin & fabric: imported spandex",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l""]",s,black,pro-support anti-smash: this sport bra teams a plunge front with a racerback and light details medium support pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash tumble dry imported polyester/spandex care: machine wash tumble dry origin & fabric: imported spandex


0.8932806324110671


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
266430,victoria sport the player by victoria sport plunge sport bra,24.5,12.99,https://www.victoriassecret.com/bras/shop-all-bras/the-player-by-victoria-sport-plunge-sport-bra-victoria-sport?productid=331806&cataloguetype=ols,victoria's secret,the player by victoria sport plunge sport bra,victoriassecret us,"pro-support, anti-smash: this sport bra teams a plunge front with a racerback and medium support. details medium support. pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning, kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash. tumble dry. imported polyester/spandex",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",s,tinted plum,pro-support anti-smash: this sport bra teams a plunge front with a racerback and medium support details medium support pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash tumble dry imported polyester/spandex
239024,victoria sport the player by victoria sport plunge sport bra,24.5,24.50,https://www.victoriassecret.com/victorias-secret-sport/shop-all/the-player-by-victoria-sport-plunge-sport-bra-victoria-sport?productid=328160&cataloguetype=ols,victoria's secret,the player by victoria sport plunge sport bra,victoriassecret us,"pro-support, anti-smash: this sport bra teams a plunge front with a racerback and light.details medium support. pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning, kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash. tumble dry. imported polyester/spandex care: machine wash. tumble dry. origin & fabric: imported spandex",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l""]",s,tinted plum,pro-support anti-smash: this sport bra teams a plunge front with a racerback and light details medium support pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash tumble dry imported polyester/spandex care: machine wash tumble dry origin & fabric: imported spandex


0.8932806324110671


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
266431,victoria sport the player by victoria sport plunge sport bra,24.5,12.99,https://www.victoriassecret.com/bras/shop-all-bras/the-player-by-victoria-sport-plunge-sport-bra-victoria-sport?productid=331806&cataloguetype=ols,victoria's secret,the player by victoria sport plunge sport bra,victoriassecret us,"pro-support, anti-smash: this sport bra teams a plunge front with a racerback and medium support. details medium support. pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning, kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash. tumble dry. imported polyester/spandex",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",s,white,pro-support anti-smash: this sport bra teams a plunge front with a racerback and medium support details medium support pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash tumble dry imported polyester/spandex
239025,victoria sport the player by victoria sport plunge sport bra,24.5,24.50,https://www.victoriassecret.com/victorias-secret-sport/shop-all/the-player-by-victoria-sport-plunge-sport-bra-victoria-sport?productid=328160&cataloguetype=ols,victoria's secret,the player by victoria sport plunge sport bra,victoriassecret us,"pro-support, anti-smash: this sport bra teams a plunge front with a racerback and light.details medium support. pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning, kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash. tumble dry. imported polyester/spandex care: machine wash. tumble dry. origin & fabric: imported spandex",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l""]",s,white,pro-support anti-smash: this sport bra teams a plunge front with a racerback and light details medium support pullover style plunge neckline racerback elastic band bottom performance & fabric perfect for spinning kickboxing and circuit training performance fabric with 4-way stretch body-wick machine wash tumble dry imported polyester/spandex care: machine wash tumble dry origin & fabric: imported spandex


In [127]:
print(del_index)
brands[brand_index] = brands[brand_index].drop(del_index)
df = df2[df2['product_name']=='very sexy lace cheeky panty']
index = df[df['mrp']>30].index.tolist()
print(index)
brands[brand_index].loc[index, 'product_name'] = 'very sexy lace cheeky panty expensive'
index = df[df['mrp']==16.5].index.tolist()
print(index)
brands[brand_index].loc[index, 'product_name'] = 'very sexy lace cheeky panty middle'
index = df[df['mrp']<16.5].index.tolist()
print(index)
brands[brand_index].loc[index, 'product_name'] = 'very sexy lace cheeky panty cheap'


[164203, 164201, 90456, 164204, 19609, 7237, 7239, 239026, 239024, 239025]
[117416, 117415, 26213, 26214, 26215, 26212]
[202859, 20684, 20687, 20689, 20686, 20688, 20690]
[188334, 188335, 188336]


In [128]:
df_main =pd.DataFrame()
for ind, brand in enumerate(brands):
    df = brand
    df2 = brand
    df = pd.DataFrame(df.groupby(['product_name','color']).size())
    df = df[df[0]>1].reset_index()
    
    df['score'] = None
    df['brand'] = brands_name[ind]
    for ind, row in df.iterrows():
        d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
        score = similar(d.iloc[0].descr_conc, d.iloc[1].descr_conc)
        df.set_value(ind,'score',score)
    df_main = df_main.append(df)
    

In [129]:
df_main

,product_name,color,0,score,brand
0,embroidered mini knickers,black,2,0.578231,Topshop
1,lace thong,dark nude,2,0.556017,Topshop
2,mini knickers by topshop bride,ivory,2,0.449275,Topshop
0,victoria's secret the ultimate maximum support sport bra,colorblock multi,2,0.0903955,Amazon Victoria's Secret
0,wireless high impact sports bra 852218,black gray,2,0.987854,Macys Wacoal


In [130]:
#Amazon Victorias secret
brand_index = 7
df2 = brands[brand_index]
df_slave = df_main[df_main['brand']==brands_name[brand_index]]
for ind, row in df_slave.iterrows():
    print(row['score'])
    d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
    display(d)

#color was wrong
index = df2[df2['pdp_url']=='https://www.amazon.com/-/dp/b072fbmxfd?th=1&psc=1'].index.tolist()
brands[brand_index].loc[index, 'color'] = 'neon multi'

0.0903954802259887


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
1966,victoria's secret the ultimate maximum support sport bra,49.49,49.49,https://www.amazon.com/-/dp/b072fbmxfd?th=1&psc=1,victorias-secret,bras,amazon us,"meet our most intuitive sports bra, with maximum support and sleek comfort for running, boxing, and high-impact work outs. wider double-lined spandex sides provide an ultra smooth shape while an encased plastic underwire flexes with you. adjustable straps can convert to crossback with front clip closures that snap into place.",4.6,58,"[ Adjust-and-go straps! Wear them classic or crossback. , Smoothing technology and perfected fit. , Extra-cushioned closure means no pain during floor exercises. , Body-Wick technology to keep you cool and dry. ]","32d , 32ddd , 34b , 34c , 34d , 34dd , 34ddd , 36c , 36d , 36dd , 36ddd , 38c , 38d , 38dd , 38ddd","34c , 34ddd",colorblock multi,meet our most intuitive sports bra with maximum support and sleek comfort for running boxing and high-impact work outs wider double-lined spandex sides provide an ultra smooth shape while an encased plastic underwire flexes with you adjustable straps can convert to crossback with front clip closures that snap into place
27371,victoria's secret the ultimate maximum support sport bra,49.49,49.49,https://www.amazon.com/-/dp/b07284hlhn?th=1&psc=1,victorias-secret,bras,amazon us,"maximum support, body-wick keeps you cool & dry, concealed, flexible underwire for cushioned support, wear it classic & crossback; with adjustable straps that attach in front, extra-cushioned adjustable back closure",4.4,65,"[ Adjust-and-go straps! Wear them classic or crossback. , Smoothing technology and perfected fit. , Extra-cushioned closure means no pain during floor exercises. , Body-Wick technology to keep you cool and dry. ]","34b , 34c , 34d , 34dd , 34ddd , 36c , 36d , 36dd , 36ddd , 38c , 38d , 38dd , 38ddd",34c,colorblock multi,maximum support body-wick keeps you cool & dry concealed flexible underwire for cushioned support wear it classic & crossback; with adjustable straps that attach in front extra-cushioned adjustable back closure


Topshop

It seems that some products have just the same product name. because there are not so many of them, we can leave them

In [131]:
brand_index = 3
df2 = brands[brand_index]
df_slave = df_main[df_main['brand']==brands_name[brand_index]]
for ind, row in df_slave.iterrows():
    print(row['score'])
    d = df2[(df2['product_name']== row['product_name'])&(df2['color']== row['color'])]
    display(d)

0.5782312925170068


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
13,embroidered mini knickers,20.0,20.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/embroidered-mini-knickers-6765296,us topshop,lingerie,topshop us,"in a black mesh design, these flattering mini knickers come in a pretty embroidered floral design. 86% polyamide, 14% elastane. machine wash.",NaN,NaN,NaN,"2,4,6,8,10,12,14","2,4,6,8,10,12",black,in a black mesh design these flattering mini knickers come in a pretty embroidered floral design 86% polyamide 14% elastane machine wash
2616,embroidered mini knickers,20.0,20.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/embroidered-mini-knickers-6125457,us topshop,lingerie,topshop us,"opt for a pretty touch when choosing your lingerie with these black mesh mini knickers complete with floral embroidery. 84% polyamide,16% elastane. machine wash.",4.0,1.0,NaN,"2,4,6,8,10,12","2,6,10",black,opt for a pretty touch when choosing your lingerie with these black mesh mini knickers complete with floral embroidery 84% polyamide 16% elastane machine wash


0.5560165975103735


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
218,lace thong,10.0,10.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/lace-thong-6688831,us topshop,lingerie,topshop us,"dark nude knickers finished with pretty lace and mesh. 89% nylon, 11% elastane. machine wash.",NaN,NaN,NaN,"2,4,6,8,10,12,14","2,4,6,8,10,12",dark nude,dark nude knickers finished with pretty lace and mesh 89% nylon 11% elastane machine wash
1189,lace thong,10.0,10.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/lace-thong-6068666,us topshop,lingerie,topshop us,"keep fitted silhouettes seam-free with this pretty nude lace thong underneath. a semi-sheer style with a low rise fit. 89% nylon,11% elastane. machine wash.",NaN,NaN,NaN,"2,4,6,8,10,12",10,dark nude,keep fitted silhouettes seam-free with this pretty nude lace thong underneath a semi-sheer style with a low rise fit 89% nylon 11% elastane machine wash


0.4492753623188406


,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
2211,mini knickers by topshop bride,20.0,20.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/mini-knickers-by-topshop-bride-6492198,us topshop,lingerie,topshop us,"elevate your bridal intimates with these pretty ivory knickers in a satin-feel. 96% polyester, 4% elastane. machine wash.",NaN,NaN,NaN,"2,4,6,8,10,12","2,6,8,10,12",ivory,elevate your bridal intimates with these pretty ivory knickers in a satin-feel 96% polyester 4% elastane machine wash
2240,mini knickers by topshop bride,22.0,22.0,http://www.topshop.com/en/tsus/product/clothing-70483/lingerie-2313852/mini-knickers-by-topshop-bride-6491987,us topshop,lingerie,topshop us,"in a beautiful ivory lace, these low-rise mini knickers come with good coverage and are perfect for wearing under your wedding dress. 100% polyester. machine wash.",NaN,NaN,NaN,"2,4,6,8,10,12","2,4,6,8,10,12",ivory,in a beautiful ivory lace these low-rise mini knickers come with good coverage and are perfect for wearing under your wedding dress 100% polyester machine wash


In [132]:
del brands_sorted

In [133]:
df_main =pd.DataFrame()
for ind, brand in enumerate(brands):
    df = brand
    df2 = brand
    df = pd.DataFrame(df.groupby(['product_name','descr_conc']).size())
    df = df[df[0]>1].reset_index()
    
    df['score'] = None
    df['brand'] = brands_name[ind]
    for ind, row in df.iterrows():
        d = df2[(df2['product_name']== row['product_name'])&(df2['descr_conc']== row['descr_conc'])]
        score = similar(d.iloc[0].color, d.iloc[1].color)
        df.set_value(ind,'score',score)
    df_main = df_main.append(df)

In [134]:
df_main[df_main['score']>0.7]

,product_name,descr_conc,0,score,brand
13,bride original rise thong,you're only a bride for so long -- make it count! show off the b-word on an original rise thong bearing sparkly flirty cursive,3,0.73913,Hanky Panky
26,i do low rise thong,"who knew two little words could mean so much? say it in style with this hanky panky low rise thong that says ""i do"" in swarovski® crystals",2,0.73913,Hanky Panky
35,maid of honor low rise thong,"repay your debt of gratitude with a maid of honor gift that'll make ladies of all ages swoon: a low rise thong bedecked with crystals spelling out ""maid of honor""",2,0.8,Hanky Panky
41,mrs. original rise thong,a new title (mrs !) deserves an irresistible new original rise thong to remind you of the good news,3,0.75,Hanky Panky
35,body by victoria unlined demi,a naturally sexy unlined shape with a lower cut that’s perfect for revealing necklines designed to disappear under curve-hugging styles lift & lining unlined underwire cups foam sling inside cup for additional support & lift straps & hooks adjustable straps can convert to crossback and snap into place for a secure hold back closure double row of hook and eye closures; sizes 34ddd 36dd-36ddd & 38d-38ddd have triple row of closures for a secure comfortable fit 4 settings to ensure a perfect fit details & fabric allover crochet lace cups super-soft bbv microfiber double layer wing for comfort imported nylon/lycra® spandex,6,0.742857,Victoria's Secret
37,body by victoria wireless bra,supersoft comfort without the wires in a plunging neckline for low-cut looks with lighter memory fit for extra support as it conforms to your curves and a smoothing u-shaped back and sides lift & lining lightly lined wireless cups straps & hooks back closure double row of hook-and-eye closures; sizes 36dd 38-40d-dd & 40c have triple row of closures for a secure comfortable fit 4 settings to ensure a perfect fit wear classic or crossback details & fabric u-shaped ballet back prevents band from riding up and offers more coverage double-lined sides for the smoothest shape imported nylon/lycra® spandex,2,0.769231,Victoria's Secret
61,cotton lingerie lace waist thong panty,this wisp of a panty is virtually invisible under clothes in supersoft stretchy cotton naturally sexy and comfy with a little lace in lots of colors and pretty prints our cotton undies ensure your lingerie drawer is anything but ordinary smooth floral lace waistband lays flat low rise no back coverage imported cotton/elastane,29,0.882353,Victoria's Secret
65,cotton lingerie mesh string bikini panty,virtually invisible under clothes this panty is designed in supersoft mesh and cotton for ultimate comfort in lots of colors and pretty prints our cotton undies ensure your lingerie drawer is anything but ordinary allover mesh low rise moderate back coverage: shows a little but not all imported nylon/cotton/spandex,5,0.75,Victoria's Secret
78,dream angels crochet lace flutter bralette,a bralette with bohemian style thanks to gorgeous ruffles in crochet lace lift & lining unlined wireless band straps & hooks adjustable removable straps pullover style details & fabric crochet lace flounce imported nylon/polyester/spandex,4,0.730159,Victoria's Secret
82,dream angels crochet lace strapless bralette,more than just a bandeau this strapless bralette goes boho with a crochet lace overlay lift & lining lightly lined for shape underwire cups straps & hooks adjustable removable straps back closure triple row of hook-and-eye closures details & fabric crochet lace overlay imported nylons/spandex/polyester,2,0.769231,Victoria's Secret


In [135]:
brand_index = 2
df2 = brands[brand_index]
df2[df2['color'].isin(['sugar cookie with chantilly lace', 'suger cookie with chantilly lace'])]

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color,descr_conc
183444,dream angels the laced up bralette,44.5,14.99,https://www.victoriassecret.com/bras/bralette/the-laced-up-bralette-dream-angels?productid=338364&cataloguetype=ols&colorcode=01r,victoria's secret,the laced-up bralette,victoriassecret us,"a stunner at every turn, from its flirty ties and scalloped lace, to a sleek racerback in sheer mesh. lift & lining unlined wireless cups straps & hooks pullover style details & fabric scalloped lace with front ties mesh racerback imported nylon/spandex",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",s,sugar cookie with chantilly lace,a stunner at every turn from its flirty ties and scalloped lace to a sleek racerback in sheer mesh lift & lining unlined wireless cups straps & hooks pullover style details & fabric scalloped lace with front ties mesh racerback imported nylon/spandex
2780,dream angels crochet lace flutter bralette,34.5,19.99,https://www.victoriassecret.com/bras/shop-all-bras/crochet-lace-flutter-bralette-dream-angels?productid=328142&cataloguetype=ols,victoria's secret,crochet lace flutter bralette,victoriassecret us,"a bralette with bohemian style, thanks to gorgeous ruffles in crochet lace.lift & lining unlined wireless band straps & hooks adjustable, removable straps pullover style details & fabric crochet lace flounce imported nylon/polyester/spandex",NaN,NaN,NaN,"[""xs"", ""s"", ""m"", ""l"", ""xl""]",s,suger cookie with chantilly lace,a bralette with bohemian style thanks to gorgeous ruffles in crochet lace lift & lining unlined wireless band straps & hooks adjustable removable straps pullover style details & fabric crochet lace flounce imported nylon/polyester/spandex
2798,dream angels crochet lace strapless bralette,44.5,19.99,https://www.victoriassecret.com/bras/shop-all-bras/crochet-lace-strapless-bralette-dream-angels?productid=335219&cataloguetype=ols,victoria's secret,crochet lace strapless bralette,victoriassecret us,"more than just a bandeau, this strapless bralette goes boho with a crochet lace overlay. lift & lining lightly lined for shape underwire cups straps & hooks adjustable, removable straps back closure triple row of hook-and-eye closures details & fabric crochet lace overlay imported nylons/spandex/polyester",NaN,NaN,NaN,"[""32b"", ""32c"", ""32d"", ""32dd"", ""34b"", ""34c"", ""34d"", ""34dd"", ""36b"", ""36c"", ""36d"", ""36dd""]",36c,sugar cookie with chantilly lace,more than just a bandeau this strapless bralette goes boho with a crochet lace overlay lift & lining lightly lined for shape underwire cups straps & hooks adjustable removable straps back closure triple row of hook-and-eye closures details & fabric crochet lace overlay imported nylons/spandex/polyester


## 1.5 Missing values
The are several reasons why a dataset can have missing vaues. People might not be careful enough while collecting the dataset, there can by some technical errors and the datset can be misleading on purpose!

In [136]:
def find_missing_values(data, name):
    df = data[list(set(data.columns.tolist()) - set(['rating', 'review_count']))]
    nulls = df.isnull().sum().sort_values(ascending=False)
    if nulls.sum() != 0:
        print(name)
        display(pd.DataFrame(nulls).transpose())
        #fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(10,4))
        #ax = nulls.plot(kind='bar', figsize = (23,5), ax = axes, title = name)

for ind, brand in enumerate(brands):
    find_missing_values(brand, brands_name[ind])

Victoria's Secret


,style_attributes,product_name,mrp,color,product_category,price,retailer,total_sizes,brand_name,description,available_size,pdp_url,descr_conc
0,3519,0,0,0,0,0,0,0,0,0,0,0,0


Topshop


,style_attributes,product_name,mrp,color,product_category,price,retailer,total_sizes,brand_name,description,available_size,pdp_url,descr_conc
0,421,0,0,0,0,0,0,0,0,0,0,0,0


Topshop has 89  products where color is missing.

## 1.6 Concatenate the brands

In [137]:
def concatenate_brands(brands_array, cols):
    concat_df = pd.DataFrame()
    for ind, brand in enumerate(brands_array):
        df = brand[cols]
        df['brand'] = brands_name[ind]
        concat_df = concat_df.append(df)
    concat_df['product_category_gen'] = concat_df['product_category']
    return concat_df

#cols = ['mrp', 'price', 'product_category', 'product_name', 'pdp_url', 'description', 'color']
#brands_info_mrp = concatenate_brands(brands_mrp, cols)

#cols = ['mrp', 'price', 'product_category', 'product_name', 'pdp_url', 'available_size', 'description', 'color']
#brands_info_price = concatenate_brands(brands_price, cols)

#cols = ['mrp', 'price', 'product_category', 'product_name', 'pdp_url', 'total_sizes', 'description', 'color']
#brands_info_mrp_sizes = concatenate_brands(brands_mrp_sizes, cols) 

cols = ['mrp', 'price', 'product_category', 'product_name', 'pdp_url', 'description', 'color', 'descr_conc']
brands_info = concatenate_brands(brands, cols) 
brands_info.index = range(len(brands_info))

## 1.6 Find bugs/weirdos

We expect that each unique item has a unique name and a discription. So, this is not true here.
Topshop has 295 unique models (according to the product_name) but there are 590 unique models (according to the product_name and description).
So, int he first phase we will find the unique products by product_name and description. In the second iteration we will find unique products by a product_name, mrp and product_category. (As in Calvin Klein 'ck id thin waistband string bikini' product can be a string or a string with a bra having the same mrp (what is also weird, right?) but the difference is in the product_category.)

In [138]:
brands_info[brands_info.pdp_url=='http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-ck-id-triangle-black']

,mrp,price,product_category,product_name,pdp_url,description,color,descr_conc,brand,product_category_gen


Lets find those products that have the same product_name but different descriptions and different product_category.

Some producs have a different mrp comparing to our dataset. <br>
For example, this [product](http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-ck-id-triangle-black) costs 32 USD on the website, in our dataset it is 20 USD



### Final weirdos 

Some websites (as we have seen already before) can sell several items on one page, for example a bra and a matching panty. The scapper will extract the description (or product name or product category) where it will be stated that it is set. In reality the set is divided into two seperate products each with its own size and price. In fact, we can get that product name and description states that is a set but we do not know if the price corresponds to a panty or a bra.

By checking the links we can understand if there is a price for the whole set or for each set's item seperate. Also, items from the sets are usually linked to already existing seperate pages of the set's parts, meaning we have them already in the dataset. Therefore we will delete all sets except for the Amazon one.


In [139]:
df = brands_info
str_panties = '(?=.*(bikini|panty|tanga|hipster|thong|short|trunk|high cut brief|knickers))'
g = df[df['product_name'].str.contains(manipul_regex(['bra','bralette', 'bandeau'])+str_panties)]
#display(g.groupby('brand').size())
#display(g.sort_values(by='product_name'))
amazon_index = df[df['pdp_url']=='https://www.amazon.com/-/dp/b06wrtg5d8?th=1&psc=1'].index.tolist()
del_index = list(set(g.index.tolist()) - set(amazon_index))
brands_info.drop(del_index, inplace = True)
print(len(del_index))

#5513 product_category == 'bra panties sets'

#questionable
#6843,6841,6840,6742,6741,6738,6736,6725, looking at the mrp it should be panties
#5970,5972,5973 (looking at the price it might really be a set or still a bra...)
#5984,5986,5987,5988,5989,5990,5993,5997 look like bras because of the price on the website has 28USD for bras
# 6097,6119, look like bras becuse of the website mrp match
# 6131,6132 seem like bra
#6251, 6256 look like bras
#6389 look like bra because of the mrp
#6911,6897,6895,6899,6774,6389 looks like bra because of the mrp


31


In [140]:
#important why black is out of the final dataset
#df = brands_info[brands_info['brand']=='Amazon Calvin Klein']
#df[(df['product_name'].str.contains("calvin klein women's modern cotton bralette and bikini set"))]
#df = amazon_klein
#df[(df['product_name'].str.contains("calvin klein women's modern cotton bralette and bikini set"))&(df['color'].isin(['black']))]

We will also explore records with 'set' in description to be sure that data will be categorized the right way.

To consider in categorization:
https://www.amazon.com/-/dp/b06wrtg5d8?th=1&psc=1 -> category : bra panties set <br>
product_name sleep set -> category : sleepwear

In [141]:
df = brands_info
g = df[df['descr_conc'].str.contains(manipul_regex(['bra','bralette', 'bandeau'])+str_panties)]
display(g.groupby('brand').size())
display(g.sort_values(by='brand'))

brand
Amazon Calvin Klein    1 
Calvin Klein           4 
Hanky Panky            11
Macys Wacoal           2 
Topshop                10
Victoria's Secret      32
dtype: int64

,mrp,price,product_category,product_name,pdp_url,description,color,descr_conc,brand,product_category_gen
5513,33.88,33.88,panties,calvin klein women's modern cotton bralette and bikini set,https://www.amazon.com/-/dp/b06wrtg5d8?th=1&psc=1,a bralette and bikini set in soft cotton for breathable comfort,white,a bralette and bikini set in soft cotton for breathable comfort,Amazon Calvin Klein,panties
290,22.00,22.00,bralettes,retro calvin short,http://www.calvinklein.us/en/womens-clothing/womens-sports-bras/womens-underwear-set-retro-black,"a retro calvin bralette + short set featuring cotton stretch fabric, an elasticized ribbed knit band and an iconic logo design.",black,a retro calvin bralette + short set featuring cotton stretch fabric an elasticized ribbed knit band and an iconic logo design,Calvin Klein,bralettes
297,22.00,22.00,panty sets,retro calvin short,http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-retro-grey,"a retro calvin bralette + short set featuring cotton stretch fabric, an elasticized ribbed knit band and an iconic logo design.",greay heather silver foil,a retro calvin bralette + short set featuring cotton stretch fabric an elasticized ribbed knit band and an iconic logo design,Calvin Klein,panty sets
361,36.00,36.00,panty sets,retro calvin bralette,http://www.calvinklein.us/en/womens-clothing/women-sets/womens-underwear-set-retro-grey,"a retro calvin bralette + short set featuring cotton stretch fabric, an elasticized ribbed knit band and an iconic logo design.",greay heather silver foil,a retro calvin bralette + short set featuring cotton stretch fabric an elasticized ribbed knit band and an iconic logo design,Calvin Klein,panty sets
366,36.00,36.00,bralettes,retro calvin bralette,http://www.calvinklein.us/en/womens-clothing/womens-sports-bras/womens-underwear-set-retro-black,"a retro calvin bralette + short set featuring cotton stretch fabric, an elasticized ribbed knit band and an iconic logo design.",black,a retro calvin bralette + short set featuring cotton stretch fabric an elasticized ribbed knit band and an iconic logo design,Calvin Klein,bralettes
1065,116.00,116.00,lingerie,40th anniversary cotton chemise,http://www.hankypanky.com/lingerie/heather-jersey-chemise-1.html,"it all started as a gift…in 1977 designer gale epstein created and bra and panty set out of hand-embroidered handkerchiefs as a birthday gift for her friend, lida orzeck. inspired by this ingenious concept, the friends became business partners and named their new company after the original design, hanky panky.",white,it all started as a gift in 1977 designer gale epstein created and bra and panty set out of hand-embroidered handkerchiefs as a birthday gift for her friend lida orzeck inspired by this ingenious concept the friends became business partners and named their new company after the original design hanky panky,Hanky Panky,lingerie
1064,116.00,116.00,lingerie,40th anniversary cotton sleep set,http://www.hankypanky.com/lingerie/heather-jersey-cotton-sleep-set.html,"it all started as a gift…in 1977 designer gale epstein created and bra and panty set out of hand-embroidered handkerchiefs as a birthday gift for her friend, lida orzeck. inspired by this ingenious concept, the friends became business partners and named their new company after the original design, hanky panky.",white,it all started as a gift in 1977 designer gale epstein created and bra and panty set out of hand-embroidered handkerchiefs as a birthday gift for her friend lida orzeck inspired by this ingenious concept the friends became business partners and named their new company after the original design hanky panky,Hanky Panky,lingerie
954,62.00,62.00,bras-and-tops,retro lace fashion bralette,http://www.hankypanky.com/bras-and-tops/bralettes/retro-lace-fashion-bralette.html,pair this retro-style bralette with its matching retro thong and you're sure to give the sexy screen sirens of old hollywood a run for their money.,marshmallow bel

In [142]:
brands_info.to_csv('cleaned_data.csv')